In [ ]:
# Scroll to the end first before you begin running the whole code'
sheet_name='SheetName'
worksheet_name = 'WorkSheetName'
key = "KeyName.json"

In [ ]:
db = 'DBName'

In [ ]:
import pyodbc
import pandas as pd

def connect(database_name):
    '''
    Connection string for ms sql to the database provided
    '''
    con = pyodbc.connect(Driver='{SQL Server}',
                         Server='Name',
                         Database='{}'.format(database_name),
#                          Trusted_Connection=True)
                         User='Name',
                         Password='password')
    return con

            #####################################
def database_table_list(database_name):
    '''
    List's the names of tables in the database
    '''
    sql = """
    SELECT * FROM information_schema.tables
    """
    database_df = pd.read_sql(sql,connect(database_name))
    connect(database_name).close()
    i = 0
    l = []
    while i < len(database_df['TABLE_NAME']):
        l.append(database_df['TABLE_NAME'][i])
        i += 1
    return l

            #####################################
def get_dataframe(table_name,database_name,index_col=None,columns=None):
    '''
    get the dataframe of a table from a database in the database
    '''
    sql = 'select * from {}'
    df = pd.read_sql(sql.format(table_name),connect(database_name),index_col=index_col,columns=columns)
    connect(database_name).close()
    return df

            #####################################
def database_table_columns(database_name):
    '''
    get the table list and columns of those tables in a dictionary in the database
    '''
    i = 0
    dic = {}
    while i < len(database_table_list(database_name)):
        table = database_table_list(database_name)[i]
        i += 1
        a = [get_dataframe(table,database_name).shape, get_dataframe(table,database_name).columns]
        dic[table]=a;
    return dic

            #####################################
def database_number_of_entries(database_name):
    '''
    get the print of all the tables and the entries in them i.e. shape of the table in the database
    '''
    col_name = database_table_columns(database_name)
    table_list = database_table_list(database_name)
    for i in range(len(col_name)):
                   print(table_list[i], ': ', col_name[table_list[i]][0])
                   i += 1
    return 'Total '+str(len(table_list))+' tables'

            #####################################
def database_table_column_text(database_name):
    '''
    Prints the Table name along with the columns of respective tables in the database
    '''
    dataset = database_table_columns(database_name)
    table_list = database_table_list(database_name)
    i=0
    while i < len(database_table_list(database_name)):
        print('\n', database_table_list(database_name)[i], ' : \n\t',end='')
        x=0
        while x < len(dataset[database_table_list(database_name)[i]][1]):
            print(dataset[database_table_list(database_name)[i]][1][x], end="  ")
            x += 1
        i += 1
        
            #####################################


# Getting all records in the dataframe

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# 

In [ ]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread

def get_client(key):
    scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
                 "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(key, scope)
    client = gspread.authorize(creds)
    return client

                    ##########################################
def sheets(sheet_name,key,worksheet_name='Sheet1'):
    client = get_client(key)

    try:
        sheet = client.open(sheet_name)
        sheet = sheet.worksheet(worksheet_name)
    except StopIteration:
        sheet = client.open(sheet_name).sheet1
    
    return sheet

                    ##########################################
def get_woksheet_list(sheet_name,key):
    client = get_client(key)
    
    sheet = client.open(sheet_name)
    sheet_list = []
    a = sheet.worksheets()
    for i in range(len(a)):
        sheet_list.append(a[i].title)
    return sheet_list

                    ##########################################
def sheets_to_dataframe(sheet_name,worksheet_name,key=key,file_name='Test'):
    
    sheet = sheets(sheet_name,key,worksheet_name)
        
    data = sheet.get_all_values()
    headers = data.pop(0)

    df = pd.DataFrame(data, columns=headers)
    writer = pd.ExcelWriter("{}.xlsx".format(file_name), engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Sheet1')
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    writer.save()
    df = pd.read_excel('./{}.xlsx'.format(file_name))
    return df


                    ##########################################
def sheets_to_datalist(sheetName, worksheet_name):
    client = get_client(key)
    sheet = client.open(sheetName).worksheet(worksheet_name)
    return sheet.get_all_values()[1:]


                    ##########################################
def sheets_add_column_names(column_list,sheet_name,key,worksheet_name='Sheet1'):

    sheet = sheets(sheet_name,key,worksheet_name)

    sheet.insert_row(column_list,1)
    return 'Done'

                    ##########################################
def sheets_add_record(rows_list,sheet_name,key,worksheet_name='Sheet1'):
    
    sheet = sheets(sheet_name,key,worksheet_name)

    sheet.append_rows(rows_list)
    print('Done')
    return None

                    ##########################################
def sheet_empty(sheet_name,key=key):
    client = get_client(key)
    sheet = client.open(sheet_name)
    c = 0
    for i in sheet.worksheets():
        try:
            sheet.del_worksheet(i)
            print(i.title+" Deleted")
            c+=1
        except:
            print(i.title+'  Left')
            pass
    sheet.add_worksheet('empty',rows=10,cols=10)
    sheet.del_worksheet(i)
    c+=1
    print(str(c-1)+' Sheets deleted')

                    ##########################################
def sheets_import_colums(tables,db,sheet_name,key=key):
    c = 0
    for table in tables:
        try:
            sheet = sheets(sheet_name,key,table)
            sheet.clear()
        except:
            print(table+" Created")
            client = get_client(key)
            sheet = client.open(sheet_name)
            sheet.add_worksheet(table,rows=10,cols=10)
        df = get_dataframe(table,db)
        cols = []
        for i in df.columns:
            cols.append(i)
        sheets_add_column_names(cols,sheet_name,key,table)
        c+=1
    print(str(c)+" table columns imported")
        
        
                    ##########################################
def sheets_import_data(tables,db,sheet_name,key=key):
    '''Fetches the table details from the sql db 
    and imports it to the respective sheets
    tables are the list of tables in sql to import'''
    c = 0
    for table in tables:
        try:
            sheet = sheets(sheet_name,key,table)
            sheet.clear()
        except:
            print(table+" Created")
            client = get_client(key)
            sheet = client.open(sheet_name)
            sheet.add_worksheet(table,rows=10,cols=10)
        df = get_dataframe(table,db)
        cols = []
        for i in df.columns:
            cols.append(i)
        sheets_add_column_names(cols,sheet_name,key,table)
        df = get_dataframe(table,db)
        row = df.values
        rows = []
        for i in range(len(row)):
            a = []
            for j in row[i]:
                a.append(j)
            rows.append(a)
        print(table)
        try:
            sheets_add_record(rows,sheet_name,key,table)
            print('')
        except TypeError:
            print(TypeError)
            print('Empty Table Created \n')
            pass
        except:
            print('api error - jason data type \n') 
            pass
        c += 1
    print('End '+str(c)+' Sheets Created \n')

# 

# Encryption Cell

In [ ]:
from cryptography.fernet import Fernet as fer

name = 'Encryption'

crypto_key = fer.generate_key()
crypto_key = crypto_key.decode()

client = get_client(key)
sheet = client.open(sheet_name)

try:
    wb = sheet.worksheet(name)
    wb.clear()
except:
    sheet.add_worksheet(name,rows=1,cols=1)
    wb = sheet.worksheet(name)
finally:
    wb.append_row([crypto_key])

        #########################################################
def encrypt_data(data,key):
    '''
        Encrypt the data
    '''
    k = fer(key)
    try:
        en = k.encrypt(bytes(data))
    except TypeError:
        en = k.encrypt(data.encode()).decode()
    return en

        ##########################################################
# print(login_pass[1])
# en = encrypt_data(login_pass[1],crypto_key)
# print(en)
# print(f(sheet.worksheet(name).get_all_values()[-1][0].encode()).decrypt(en.encode()).decode())    #en

# 

In [ ]:
from random import choice, sample, random,randrange
import pandas as pd
from datetime import date

def segregate_vilagers(boy,girl,man,woman,husband,wife,widowed,others):
    m3 = 0
    m4 = 0
    m5 = 0
    for i in range(len(house)):
        h = house.pop(randrange(len(house)))
        if choice(range(10)) in [3,6]:                     #3 members
            m3+=1
            if choice(range(10)) in [0,1]:                           #widowed
                w = list(widowed.pop(randrange(len(widowed))))
                x,y,z,a,b = w
                w.append(h)
                house_res_id.append((h,y))
                final.append(w)
                if a>45:
                    if choice(range(10))==0:
                        g = list(girl.pop(randrange(len(girl))))
                        g1 = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g1.append(h)
                        final.append(g)
                        final.append(g1)
                    elif choice(range(10))==1:
                        b = list(boy.pop(randrange(len(boy))))
                        wo = list(woman.pop(randrange(len(woman))))
                        b.append(h)
                        wo.append(h)
                        final.append(b)
                        final.append(wo)
                    elif choice(range(10))==2:
                        b = list(boy.pop(randrange(len(boy))))
                        b1 = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b1.append(h)
                        final.append(b)
                        final.append(b1)
                    elif choice(range(10))==3:
                        b = list(boy.pop(randrange(len(boy))))
                        m = list(man.pop(randrange(len(man))))
                        b.append(h)
                        m.append(h)
                        final.append(b)
                        final.append(m)
                    elif choice(range(10))==4:
                        g = list(girl.pop(randrange(len(girl))))
                        wo = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        wo.append(h)
                        final.append(g)
                        final.append(wo)
                    elif choice(range(10))==5:
                        g = list(girl.pop(randrange(len(girl))))
                        m = list(man.pop(randrange(len(man))))
                        g.append(h)
                        m.append(h)
                        final.append(g)
                        final.append(m)
                    elif choice(range(10))==6:
                        m = list(man.pop(randrange(len(man))))
                        m1 = list(man.pop(randrange(len(man))))
                        m1.append(h)
                        m.append(h)
                        final.append(m1)
                        final.append(m)
                    elif choice(range(10))==7:
                        wo = list(woman.pop(randrange(len(woman))))
                        wo1 = list(woman.pop(randrange(len(woman))))
                        wo.append(h)
                        wo1.append(h)
                        final.append(wo)
                        final.append(wo1)
                    elif choice(range(10))==7:
                        m = list(man.pop(randrange(len(man))))
                        wo = list(woman.pop(randrange(len(woman))))
                        wo.append(h)
                        m.append(h)
                        final.append(wo)
                        final.append(m)
                    else:
                        b = list(boy.pop(randrange(len(boy))))
                        g = list(girl.pop(randrange(len(girl))))
                        b.append(h)
                        g.append(h)
                        final.append(g)
                        final.append(b)

                    ###############################

                else:
                    if choice(range(10))==0:
                        g = list(girl.pop(randrange(len(girl))))
                        g1 = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g1.append(h)
                        final.append(g)
                        final.append(g1)
                    elif choice(range(10))==2:
                        b = list(boy.pop(randrange(len(boy))))
                        b1 = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b1.append(h)
                        final.append(b)
                        final.append(b1)
                    else:
                        b = list(boy.pop(randrange(len(boy))))
                        g = list(girl.pop(randrange(len(girl))))
                        b.append(h)
                        g.append(h)
                        final.append(g)
                        final.append(b)

                        #############################

            elif choice(range(10)) in [5,4]:                           #others
                o = list(others.pop(randrange(len(others))))
                x,y,z,a,b = o
                o.append(h)
                house_res_id.append((h,y))
                final.append(o)
                if a>45:
                    if choice(range(10))==0:
                        g = list(girl.pop(randrange(len(girl))))
                        g1 = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g1.append(h)
                        final.append(g)
                        final.append(g1)
                    elif choice(range(10))==1:
                        b = list(boy.pop(randrange(len(boy))))
                        wo = list(woman.pop(randrange(len(woman))))
                        b.append(h)
                        wo.append(h)
                        final.append(b)
                        final.append(wo)
                    elif choice(range(10))==2:
                        b = list(boy.pop(randrange(len(boy))))
                        b1 = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b1.append(h)
                        final.append(b)
                        final.append(b1)
                    elif choice(range(10))==3:
                        b = list(boy.pop(randrange(len(boy))))
                        m = list(man.pop(randrange(len(man))))
                        b.append(h)
                        m.append(h)
                        final.append(b)
                        final.append(m)
                    elif choice(range(10))==4:
                        g = list(girl.pop(randrange(len(girl))))
                        wo = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        wo.append(h)
                        final.append(g)
                        final.append(wo)
                    elif choice(range(10))==5:
                        g = list(girl.pop(randrange(len(girl))))
                        m = list(man.pop(randrange(len(man))))
                        g.append(h)
                        m.append(h)
                        final.append(g)
                        final.append(m)
                    elif choice(range(10))==6:
                        m = list(man.pop(randrange(len(man))))
                        m1 = list(man.pop(randrange(len(man))))
                        m1.append(h)
                        m.append(h)
                        final.append(m1)
                        final.append(m)
                    elif choice(range(10))==7:
                        wo = list(woman.pop(randrange(len(woman))))
                        wo1 = list(woman.pop(randrange(len(woman))))
                        wo.append(h)
                        wo1.append(h)
                        final.append(wo)
                        final.append(wo1)
                    elif choice(range(10))==7:
                        m = list(man.pop(randrange(len(man))))
                        wo = list(woman.pop(randrange(len(woman))))
                        wo.append(h)
                        m.append(h)
                        final.append(wo)
                        final.append(m)
                    else:
                        b = list(boy.pop(randrange(len(boy))))
                        g = list(girl.pop(randrange(len(girl))))
                        b.append(h)
                        g.append(h)
                        final.append(g)
                        final.append(b)

                    ###############################

                else:
                    if choice(range(10))==0:
                        g = list(girl.pop(randrange(len(girl))))
                        g1 = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g1.append(h)
                        final.append(g)
                        final.append(g1)
                    elif choice(range(10))==2:
                        b = list(boy.pop(randrange(len(boy))))
                        b1 = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b1.append(h)
                        final.append(b)
                        final.append(b1)
                    else:
                        b = list(boy.pop(randrange(len(boy))))
                        g = list(girl.pop(randrange(len(girl))))
                        b.append(h)
                        g.append(h)
                        final.append(g)
                        final.append(b)
                        #############################

            else:                                               #married
                hu = list(husband.pop(randrange(len(husband))))
                x,y,z,a,b = hu
                hu.append(h)
                house_res_id.append((h,y))
                wi = list(wife.pop(randrange(len(wife))))
                x1,y1,z1,a1,b1 = wi
                wi.append(h)
                final.append(hu)
                final.append(wi)
                res_spouse_id.append((y,y1))
                res_spouse_id.append((y1,y))
                if a<45:
                    if choice(range(10)) in [0,2,4,6,8]:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        final.append(g)
                    else:
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        final.append(b)
                else:
                    if choice(range(10)) in [0,2]:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        final.append(g)
                    elif choice(range(10)) in [1,3]:
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        final.append(b)
                    elif choice(range(10)) in [4,6,8]:
                        w = list(woman.pop(randrange(len(woman))))
                        w.append(h)
                        final.append(w)
                    else:
                        m = list(man.pop(randrange(len(man))))
                        m.append(h)
                        final.append(m)

    ###################################################################

        elif choice(range(10)) in [0,1,2]:                     #5 members
            m5+=1
            hu = list(husband.pop(randrange(len(husband))))
            x,y,z,a,b = hu
            hu.append(h)
            house_res_id.append((h,y))
            wi = list(wife.pop(randrange(len(wife))))
            x1,y1,z1,a1,b1 = wi
            wi.append(h)
            final.append(hu)
            final.append(wi)
            res_spouse_id.append((y,y1))
            res_spouse_id.append((y1,y))


            if a>45:
                if choice(range(10))==0:
                    g = list(girl.pop(randrange(len(girl))))
                    g1 = list(girl.pop(randrange(len(girl))))
                    m = list(man.pop(randrange(len(man))))
                    g.append(h)
                    g1.append(h)
                    m.append(h)
                    final.append(g)
                    final.append(m)
                    final.append(g1)
                elif choice(range(10))==1:
                    b = list(boy.pop(randrange(len(boy))))
                    wo = list(woman.pop(randrange(len(woman))))
                    wo1 = list(woman.pop(randrange(len(woman))))
                    b.append(h)
                    wo.append(h)
                    wo1.append(h)
                    final.append(b)
                    final.append(wo)
                    final.append(wo1)
                elif choice(range(10))==2:
                    b = list(boy.pop(randrange(len(boy))))
                    wo = list(woman.pop(randrange(len(woman))))
                    b1 = list(boy.pop(randrange(len(boy))))
                    b.append(h)
                    wo.append(h)
                    b1.append(h)
                    final.append(b)
                    final.append(wo)
                    final.append(b1)
                elif choice(range(10))==3:
                    b = list(boy.pop(randrange(len(boy))))
                    m = list(man.pop(randrange(len(man))))
                    m1 = list(man.pop(randrange(len(man))))
                    b.append(h)
                    m.append(h)
                    m1.append(h)
                    final.append(b)
                    final.append(m)
                    final.append(m1)
                elif choice(range(10))==4:
                    g = list(girl.pop(randrange(len(girl))))
                    b = list(boy.pop(randrange(len(boy))))
                    wo = list(woman.pop(randrange(len(woman))))
                    g.append(h)
                    b.append(h)
                    wo.append(h)
                    final.append(g)
                    final.append(b)
                    final.append(wo)
                elif choice(range(10))==5:
                    g = list(girl.pop(randrange(len(girl))))
                    m = list(man.pop(randrange(len(man))))
                    m1 = list(man.pop(randrange(len(man))))
                    g.append(h)
                    m1.append(h)
                    m.append(h)
                    final.append(g)
                    final.append(m1)
                    final.append(m)
                elif choice(range(10))==6:
                    m = list(man.pop(randrange(len(man))))
                    m1 = list(man.pop(randrange(len(man))))
                    g = list(girl.pop(randrange(len(girl))))
                    m1.append(h)
                    m.append(h)
                    g.append(h)
                    final.append(m1)
                    final.append(m)
                    final.append(g)
                elif choice(range(10))==7:
                    wo = list(woman.pop(randrange(len(woman))))
                    wo1 = list(woman.pop(randrange(len(woman))))
                    b = list(boy.pop(randrange(len(boy))))
                    wo.append(h)
                    b.append(h)
                    wo1.append(h)
                    final.append(wo)
                    final.append(b)
                    final.append(wo1)
                elif choice(range(10))==7:
                    m = list(man.pop(randrange(len(man))))
                    wo = list(woman.pop(randrange(len(woman))))
                    g = list(girl.pop(randrange(len(girl))))
                    wo.append(h)
                    m.append(h)
                    g.append(h)
                    final.append(wo)
                    final.append(m)
                    final.append(g)
                else:
                    b = list(boy.pop(randrange(len(boy))))
                    g = list(girl.pop(randrange(len(girl))))
                    wo = list(woman.pop(randrange(len(woman))))
                    b.append(h)
                    g.append(h)
                    wo.append(h)
                    final.append(g)
                    final.append(wo)
                    final.append(b)

                ###############################

            else:
                if choice(range(10)) in [1,0,8]:
                    g = list(girl.pop(randrange(len(girl))))
                    g1 = list(girl.pop(randrange(len(girl))))
                    b = list(boy.pop(randrange(len(boy))))
                    g.append(h)
                    b.append(h)
                    g1.append(h)
                    final.append(g)
                    final.append(b)
                    final.append(g1)
                elif choice(range(10)) in [4,5,9]:
                    b = list(boy.pop(randrange(len(boy))))
                    b1 = list(boy.pop(randrange(len(boy))))
                    b2 = list(boy.pop(randrange(len(boy))))
                    b.append(h)
                    b1.append(h)
                    b2.append(h)
                    final.append(b)
                    final.append(b1)
                    final.append(b2)
                elif choice(range(10)) in [6,7]:
                    g = list(girl.pop(randrange(len(girl))))
                    g1 = list(girl.pop(randrange(len(girl))))
                    g2 = list(girl.pop(randrange(len(girl))))
                    g.append(h)
                    g1.append(h)
                    g2.append(h)
                    final.append(g)
                    final.append(g1)
                    final.append(g2)
                else:
                    b = list(boy.pop(randrange(len(boy))))
                    b1 = list(boy.pop(randrange(len(boy))))
                    g = list(girl.pop(randrange(len(girl))))
                    b.append(h)
                    g.append(h)
                    b1.append(h)
                    final.append(b)
                    final.append(g)
                    final.append(b1)

                #############################

        else:                                               #4 members
            m5+=1
            hu = list(husband.pop(randrange(len(husband))))
            x,y,z,a,b = hu
            hu.append(h)
            house_res_id.append((h,y))
            wi = list(wife.pop(randrange(len(wife))))
            x1,y1,z1,a1,b1 = wi
            wi.append(h)
            final.append(hu)
            final.append(wi)
            res_spouse_id.append((y,y1))
            res_spouse_id.append((y1,y))
            if a>45:
                if choice(range(10))==0:
                    g = list(girl.pop(randrange(len(girl))))
                    g1 = list(girl.pop(randrange(len(girl))))
                    g.append(h)
                    g1.append(h)
                    final.append(g)
                    final.append(g1)
                elif choice(range(10))==1:
                    b = list(boy.pop(randrange(len(boy))))
                    wo = list(woman.pop(randrange(len(woman))))
                    b.append(h)
                    wo.append(h)
                    final.append(b)
                    final.append(wo)
                elif choice(range(10))==2:
                    b = list(boy.pop(randrange(len(boy))))
                    b1 = list(boy.pop(randrange(len(boy))))
                    b.append(h)
                    b1.append(h)
                    final.append(b)
                    final.append(b1)
                elif choice(range(10))==3:
                    b = list(boy.pop(randrange(len(boy))))
                    m = list(man.pop(randrange(len(man))))
                    b.append(h)
                    m.append(h)
                    final.append(b)
                    final.append(m)
                elif choice(range(10))==4:
                    g = list(girl.pop(randrange(len(girl))))
                    wo = list(woman.pop(randrange(len(woman))))
                    g.append(h)
                    wo.append(h)
                    final.append(g)
                    final.append(wo)
                elif choice(range(10))==5:
                    g = list(girl.pop(randrange(len(girl))))
                    m = list(man.pop(randrange(len(man))))
                    g.append(h)
                    m.append(h)
                    final.append(g)
                    final.append(m)
                elif choice(range(10))==6:
                    m = list(man.pop(randrange(len(man))))
                    m1 = list(man.pop(randrange(len(man))))
                    m1.append(h)
                    m.append(h)
                    final.append(m1)
                    final.append(m)
                elif choice(range(10))==7:
                    wo = list(woman.pop(randrange(len(woman))))
                    wo1 = list(woman.pop(randrange(len(woman))))
                    wo.append(h)
                    wo1.append(h)
                    final.append(wo)
                    final.append(wo1)
                elif choice(range(10))==7:
                    m = list(man.pop(randrange(len(man))))
                    wo = list(woman.pop(randrange(len(woman))))
                    wo.append(h)
                    m.append(h)
                    final.append(wo)
                    final.append(m)
                else:
                    b = list(boy.pop(randrange(len(boy))))
                    g = list(girl.pop(randrange(len(girl))))
                    b.append(h)
                    g.append(h)
                    final.append(g)
                    final.append(b)

                ###############################

            else:
                if choice(range(10))==0:
                    g = list(girl.pop(randrange(len(girl))))
                    g1 = list(girl.pop(randrange(len(girl))))
                    g.append(h)
                    g1.append(h)
                    final.append(g)
                    final.append(g1)
                elif choice(range(10))==2:
                    b = list(boy.pop(randrange(len(boy))))
                    b1 = list(boy.pop(randrange(len(boy))))
                    b.append(h)
                    b1.append(h)
                    final.append(b)
                    final.append(b1)
                else:
                    b = list(boy.pop(randrange(len(boy))))
                    g = list(girl.pop(randrange(len(girl))))
                    b.append(h)
                    g.append(h)
                    final.append(g)
                    final.append(b)
    return 'Done'
                    
                    ##############################################
def calculateAge(born):
    '''
    Calculate age
    '''
    today = date.today()
    try: 
        birthday = born.replace(year = today.year)
  
    # raised when birth date is February 29
    # and the current year is not a leap year
    except ValueError: 
        birthday = born.replace(year = today.year,
                  month = born.month + 1, day = 1)
  
    if birthday > today:
        return today.year - born.year - 1
    else:
        return today.year - born.year          
# # Driver code
# print(calculateAge(d[1]), "years")

            #####################################

def random_dates(start_year,end_year,value_count=3000):
    '''
    creates a random birthday/date within a given range of years, default count of values 3000
    '''
    dates = []
    y = range(start_year,end_year+1)
    years=[]
    for i in y:
        years.append(i)
    i = 0
    for i in range(0,value_count):
        month = choice(range(1,13))
        day = choice(range(1,32))
        year = choice(years)
        today = date.today()
        if year == today.year:                                             #Current year
            if month > today.month:
                month = choice(range(1,today.month+1))
            if month == today.month and day > today.day:
                day = choice(range(1,today.day+1))
        if (year%4) == 0:                                                 #leapyear
            if month == 2:
                if day in [30,31]:
                    day = 29
                    dates.append(str(month)+'/'+str(day)+'/'+str(year))
            elif month in [4,6,9,11]:
                if day == 31:
                    day=choice(range(1,31))
                    dates.append(str(month)+'/'+str(day)+'/'+str(year))
            else:
                dates.append(str(month)+'/'+str(day)+'/'+str(year))    
        else:
            if month == 2:
                if day in [29,30,31]:                                       #feb
                    day = 28
                    dates.append(str(month)+'/'+str(day)+'/'+str(year))    
            elif month in [4,6,9,11]:
                if day == 31:
                    day=choice(range(1,31))
                    dates.append(str(month)+'/'+str(day)+'/'+str(year))
            else:
                dates.append(str(month)+'/'+str(day)+'/'+str(year))
        i+=1
    return dates


            #####################################
def random_death_dates(start_year,end_year,value_count=2000,rand=8,acceptable_list=[1]):
    '''
    creates a random death date within a given range of years, default count of values 2000
    rand = 8 gives near 300-350,10 near 250,290 near , 50 near 70-80 values
    '''
    dates=[]
    y = range(start_year,end_year+1)
    i=1
    years=[]
    count_no = 0
    randomizer = range(0,rand)
    today = date.today()    
    for i in y:
        years.append(i)
        i+=1
    i = 0
    for i in range(0,value_count):
        month = choice(range(1,13))
        day = choice(range(1,32))
        year = choice(years)
        if year == today.year:                                       #Current Year
            if month > today.month:
                month = choice(range(1,today.month+1))
            if month == today.month and day > today.day:
                day = choice(range(1,today.day+1))
        if choice(randomizer) in acceptable_list:
            count_no+=1
            if (year%4) == 0:                                                 #leapyear
                if month == 2:
                    if day in [30,31]:
                        day = 29
                        dates.append(str(month)+'/'+str(day)+'/'+str(year))
                elif month in [4,6,9,11]:
                    if day == 31:
                        day=choice(range(1,31))
                        dates.append(str(month)+'/'+str(day)+'/'+str(year))
                else:
                    dates.append(str(month)+'/'+str(day)+'/'+str(year))    
            else:
                if month == 2:
                    if day in [29,30,31]:                                       #feb
                        day = 28
                        dates.append(str(month)+'/'+str(day)+'/'+str(year))    
                elif month in [4,6,9,11]:
                    if day == 31:
                        day=choice(range(1,31))
                        dates.append(str(month)+'/'+str(day)+'/'+str(year))
                else:
                    dates.append(str(month)+'/'+str(day)+'/'+str(year))        
        else:
            dates.append('')
        i+=1
    return dates,count_no

            #####################################

def random_time(start,end,count = 3000):

    time = []
    h = range(start,end)
    m = range(0,60)
    s = range(0,60)    
    ms = range(0,1000)
    for i in range(count):
        hr = str(choice(h))
        mins = str(choice(m))
        sec = str(choice(s))
        milisec = str(choice(ms))
        if len(milisec) == 2:
            milisec= '0'+milisec
        if len(milisec) == 1:
            milisec= '00'+milisec
        if len(mins) == 1:
            mins = '0'+mins
        if len(sec) == 1:
            sec = '0'+sec
        time.append(hr+':'+mins+':'+sec)#+'.'+milisec)
    return time

            #####################################

def random_names(file_path,name_count=3000):
#     df = pd.read_csv('./data/names.txt')
    df = pd.read_csv(file_path)
    df = df.dropna()
    a=[]
    i=0
    for i in range(0,name_count):
        x = choice(df.values)[0]
        a.append(x)
#         print(x)
        i+=1
    return a

            #####################################
def random_first_name(full_name_list):
    a = 0
    l = []
    for i in full_name_list:
        i = i.split(' ')
        i = i[0]
        i = i.split('@')[0]
        if i == 'mr.' or i == 'mo.' or i == 'md.' or i == 'mohd.' or i == 'mohd' or i == 'mho' or i == 'mho.' or i == 'sh.' or i == 'dr.' or i == 'ct.' or i == 'ct' or i == 'md' or i == 'sh' or i == 'b' or i == 'mh' or i == 'm' or i == 'na' or i == 'km' or i == 'vi' or i== 'ku.' or i== 'km0' or i== 'smt' or i== 'mrs' or i== 'km.' or i == 'smt.' or i == 'n.' or i == 'dd' or i == 'modh.' or i == 'modh' or i == "p.":
            pass
        else:
            a += 1
            l.append(i)
    print(a)
    l = sample(l,len(l))
    return l

            #####################################
def random_last_name(full_name_list):
    a = 0
    l = []
    for i in full_name_list:
        i = i.split(" ")
        if len(i)<2:
            pass
        elif i[1]=="":
            pass
        elif i[1] == 's/o' or i[1] == 's' or i[1] == 'v' or i[1] == '@' or i[1] == 'u' or i[1] == '@' or i[1] == '&' or i[1] == 'd/o' or i[1] == 'w/o' :
            pass
        elif i[1].endswith('@') or i[1].startswith('@'):
            a+=1
            l.append(i[1].strip('@'))
        else:
            a+=1
            l.append(i[1])
    l = sample(l,len(l))
    print(a)
    return l

            #####################################
def sequence_word(word,length):
    i=1
    a=[]
    for i in range(1,length+1):
        a.append(word+str(i))
    return a

            #####################################

            #####################################
def random_village_region_street_house(tcount=3000):
    h = sample(range(1,27),26)
    s = sequence_word('street',6)
    r = sequence_word('region',7)
    v = sequence_word('village',4)
    var='A'
    alphabets=[]
    alphabets=sample([(chr(ord(var)+i)) for i in range(26)],26)
    village = []
    region = []
    street = []
    house = []
    ''' VILLAGE '''
    i = 0
    while i < 4:
        j=0
        while j < 757:
            village.append(v[i])
            j+=1
        i+=1
    village = sample(village,len(village))
    ''' REGION '''
    i=0
    x=[]
    while i < 7:
        j=0
        while j < 108:
            x.append(r[i])
            j+=1
        i+=1
    region.extend(sample(x,len(x)))
    region.extend(sample(x,len(x)))
    region.extend(sample(x,len(x)))
    region.extend(sample(x,len(x)))
    ''' STREET '''
    i=0
    x=[]
    while i<6:
        j=0
        while j < 18:
            x.append(s[i])
            j+=1
        i+=1
    i=0
    while i<28:
        street.extend(sample(x,len(x)))
        i+=1
    '''HOUSE'''
    j=0
    x=[]
    while j<tcount//28+1:
        i=choice(range(0,25))
        x.append(str(h[i])+alphabets[i])
        j+=1
    i=0
    while i<28:
        house.extend(sample(x,len(x)))
        i+=1

    return (village), (region), (street), (house)

            #####################################
def random_marital(count=3000):
    
    '''
    i=1
    while i <1000:
        if i%14==0:
            print(random_marital(3028).count(''),end='\n')
        else:
            print(random_marital(3028).count(''),end=' ')
        i+=1
    '''
#     m = ('Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single '
#          ,'Married ','Married ','Married ','Married ','Married ','Married ','Married ','Married ','Married ','Married ','Married '
#          ,'Widowed ','Widowed ','Widowed ','Widowed ','Widowed ','Widowed ' 
#          ,'Separated ','Separated '
#          ,'Divorced','Divorced',)
    m = (1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
         ,2,2,2,2,2,2,2,2,2,2,2
         ,3,3,3,3,3,3 
         ,4
         ,5,)    
    i=0
    x=[]
    for i in range(count):
        x.append(choice(m))
    return x

            #####################################
def random_ratio_marital(count=3000):
    i = 1
    while i<count:
        x=random_marital(100)
        if i%5==0:
            print(x.count('Single '),x.count('Married '),x.count('Widowed '),x.count('Separated '),x.count('Divorced'),end='\n')
        else:
            print(x.count('Single '),x.count('Married '),x.count('Widowed '),x.count('Separated '),x.count('Divorced'),end='    ')
        i+=1
    return "Count = "+str(count)

            #####################################
def random_designations(count=3000):
    
    ''' 
    i=1
    while i <1000:
        if i%14==0:
            print(3028-random_designations(3028).count(''),end='\n')
        else:
            print(3028-random_designations(3028).count(''),end=' ')
        i+=1
    '''
    d = (
        'Data Entry Operator','','','','','','','','','','',
        'Computer Operator','','','','','','','','','',
        'Accountant ','','','','','','','','','','',
        'Computer Operator/Data Entry Operator','','','','','','','','','','',
        'Technical Person','','','','','','','','','','',
        'Technical Assistant','','','','','','','','','','',
        'Overseer','','','','','','','','','',
        'Operator','','','','','','','','','','',
        'Office Clerk','','','','','','','','','','',
        'Accredited Engineer','','','','','','','','','','',
        'Auditor','','','','','','','','','','',
        'Accountant DEO','','','','','','','','','',
        'Electrical Technician','','','','','','','','','','',
        'Lower Division Clerk','','','','','','','','','','',
        'Computer Hardware Engineer','','','','','','','','','','',
        'VAT Resources','','','','','','','','','','',
        'CAR Driver','','','','','','','','','','',
        'Clerk Typist','','','','','','','','','',
        'Social Media Executive','','','','','','','','','','',
        'Attendant','','','','','','','','','','',
        'Data Processing Officer Government Contract job','','','','','','','','','','',
        'Back Office Computer Operator','','','','','','','','','',
        'Engineer','','','','','','','','','','',
        'Executive Accountant','','','','','','','','','','',
        'Site Engineer','','','','','','','','','','',
        'Staff Accountant','','','','','','','','','','',
        'Executive Assistant','','','','','','','','','','',
        'fresher','','','','','','','','','','',
        'Senior Accountant\r','','','','','','','','','','',
        'Data Operator','','','','','','','','','','',
        'Document Controller','','','','','','','','','','',
        'Electrician','','','','','','','','','','',
        'Deputy Project Manager','','','','','','','','','','',
        'Project Assistant','','','','','','','','','','',
        'Legal Advisor','','','','','','','','','','',
        'Junior Engineer Civil','','','','','','','','','','',
        'Clerk','','','','','','','','','','',
        'Assistant','','','','','','','','','','',
        'GIS Engineer','','','','','','','','','','',
        'Office Assistant','','','','','','','','','','',
    )
#     d = (
#         'Data Entry Operator',
#         'Computer Operator',
#         'Accountant ',
#         'Computer Operator/Data Entry Operator',
#         'Technical Person',
#         'Technical Assistant',
#         'Overseer',
#         'Operator',
#         'Office Clerk',
#         'Accredited Engineer',
#         'Auditor',
#         'Accountant DEO',
#         'Electrical Technician',
#         'Lower Division Clerk',
#         'Computer Hardware Engineer',
#         'VAT Resources',
#         'CAR Driver',
#         'Clerk Typist',
#         'Social Media Executive',
#         'Attendant',
#         'Data Processing Officer Government Contract job',
#         'Back Office Computer Operator',
#         'Engineer',
#         'Executive Accountant',
#         'Site Engineer',
#         'Staff Accountant',
#         'Executive Assistant',
#         'fresher',
#         'Senior Accountant\r',
#         'Data Operator',
#         'Document Controller',
#         'Electrician',
#         'Deputy Project Manager',
#         'Project Assistant',
#         'Legal Advisor',
#         'Junior Engineer Civil',
#         'Clerk',
#         'Assistant',
#         'GIS Engineer',
#         'Office Assistant'
#     )
    
    i=0
    x=[]
    c=0
    for i in range(count):
        if choice(d)=="":
            x.append(choice(d))
        else:
            c+=1
            x.append(choice(d))
    return x,c


            #####################################
def random_disability(count=3000):
    
    '''
    i=1
    while i <1000:
        if i%14==0:
            print(random_disability(3028).count(''),end='\n')
        else:
            print(random_disability(3028).count(''),end=' ')
        i+=1
    '''
#     m = ('','','',''
#          ,'Blindness','','',''
#          ,'Cerebral Palsy','','',''
#          ,'Low Vision','','',''
#          ,'Locomotor Disability','','',''
#          ,'Leprosy -Cured','','',''
#          ,'Mental Retardation','','',''
#          ,'Mental Illness','','',''
#          ,'Hearing Impairment','','',''
#         )
    m = (1,1,1,1
         ,2,1,1,1
         ,3,1,1,1
         ,4,1,1,1
         ,5,1,1,1
         ,6,1,1,1
         ,7,1,1,1
         ,8,1,1,1
         ,9,1,1,1
        )
    
    i=0
    x=[]
    for i in range(count):
        x.append(choice(m))
    return x

            #####################################
def random_sex(count=3000):
    
    '''
    i=1
    r = random_sex()
    while i <1000:
        if i%9==0:
            print(r.count('Male'),':',r.count('Female'),end='\n')
        else:
            print(r.count('Male'),':',r.count('Female'),end='----')
        i+=1
    '''
    m = (
        'Male','Male','Male','Male'
        ,'Female','Female','Female'
    )
    #m = ('Male','Female')
    
    i=0
    x=[]
    for i in range(count):
        x.append(choice(m))
    return x

            #####################################
def random_year(start,end,value_count=3000):
    y=range(start,end+1)
    year=[]
    r=[]
    for i in y:
        year.append(i)
    i=0
    while i<value_count:
        r.append(choice(year))
        i+=1
    return r

            #####################################
def random_area(start,end,value_count=3000):
    min = start
    max = end
    i=0
    r=[]
    while i<value_count:
        f = min + (max-min)*random()
        r.append(float("{:.3f}".format(f)))
        i+=1
    return r

            #####################################


# 

In [ ]:
male='csvfilepath'
female='csvfilepath'

# 

count the number of each unique value


df.value_counts()

In [ ]:
name_date_count = 100000

In [ ]:
death_count = 15000

In [ ]:
ratio = pd.read_csv("./village populas.csv",index_col=(4))['ratio'].tolist()

In [ ]:
tcount = sum(ratio)

In [ ]:
# ratio = [100,50,30,40,20,50,30,40,20,40,20,30,40,58,100,24,5]
# len(ratio)


#  Tcount is overwritten in the village section Managed by ratio

In [ ]:
database_table_list(db)

In [ ]:
try:
    gp_names = sheets_to_dataframe(sheet_name,'GramPanchayat')['VillageName'].tolist()
    print("Sheet Available")
except:
    print('Worksheet Does not exist')
    df = pd.read_csv('./GramPanchayat.csv')
    table_data = []
    table_data.append(df.columns.tolist())
    table_data.extend(df.values.tolist())
    sheet.add_worksheet('grampanchayat',rows=10,cols=10)
    wb = sheet.worksheet('grampanchayat')
    wb.append_rows(table_data)
    gp_names = df['VillageName'].tolist()

In [ ]:
gp_contact = []
num = [0,1,2,3,4,5,6,7,8,9]
i = 0
while i<len(gp_names):
    i+=1
    if choice(num) in [1,3,4]:
        ph = '9'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    elif choice(num) in [2,5,7]:
        ph = '7'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    elif choice(num) in [6,8,9]:
        ph = '8'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    else:
        ph = '0'+'2'+'2'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    gp_contact.append(int(ph))
for i in gp_contact:
    print(i)
len(gp_contact)

In [ ]:
gp_email = []
n = [0,1,2,3,4,5,6,7,8,9]
for i in gp_names:
    if choice(n) in [1,2,3,4,6,7,8]:
        mail = i.lower()+'@gmail.com'
    elif choice(n) in [0,9]:
        mail = i.lower()+'@yahoo.com'
    elif choice(n)==5:
        mail = i.lower()+'@outlook.com'
    else:
        mail = i.lower()+'@reddit.com'
    gp_email.append(mail) 
for i in gp_email:
    print(i)
len(gp_email)

# 

In [ ]:
try:
    village_names = sheets_to_dataframe(sheet_name,'Village')['VillageName'].tolist()
    print('Worksheet Available')
except:
    print('error')
    df = pd.read_csv('./Village.csv')
    table_data = []
    table_data.append(df.columns.tolist())
    table_data.extend(df.values.tolist())
    sheet.add_worksheet('village',rows=10,cols=10)
    wb = sheet.worksheet('village')
    wb.append_rows(table_data)
    village_names = df['VillageName'].tolist()

In [ ]:
v_date = []
dates = random_dates(2010,date.today().year,len(village_names)+20)
for i in dates:
    v_date.append(i)
for i in v_date:
    print(i)
len(v_date)

In [ ]:
v_ph = []
num = [0,1,2,3,4,5,6,7,8,9]
i = 0
while i<len(village_names):
    i+=1
    if choice(num) in [1,3,4]:
        ph = '9'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    elif choice(num) in [2,5,7]:
        ph = '7'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    elif choice(num) in [6,8,9]:
        ph = '8'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    else:
        ph = '0'+'2'+'2'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    v_ph.append(int(ph))
for i in v_ph:
    print(i)
len(v_ph)

In [ ]:
village_email = []
n = [0,1,2,3,4,5,6,7,8,9]
for i in village_names:
    if choice(n) in [1,2,3,4,6,7,8]:
        mail = i.lower()+'@gmail.com'
    elif choice(n) in [0,9]:
        mail = i.lower()+'@yahoo.com'
    elif choice(n)==5:
        mail = i.lower()+'@outlook.com'
    else:
        mail = i.lower()+'@reddit.com'
    village_email.append(mail) 
for i in village_email:
    print(i)
len(village_email)

# 

In [ ]:
list_resident = []

In [ ]:
c = 0 
for i in ratio:
    c+=i
c

# Replacing Tcount here

In [ ]:
rv_id = []
# a = range(1,6)
a = 1
for i in ratio:
    for j in range(i):
        rv_id.append(a)
    a+=1
for i in range(1,18):
    print(str(i)+" "+str(rv_id.count(i)),end='---')
# tcount = len(rv_id)
rv_id = sample(rv_id,len(rv_id))
rv_id = sample(rv_id,len(rv_id))
rv_id = sample(rv_id,len(rv_id))
len(rv_id)

In [ ]:
df = pd.read_csv('./village.csv')[['ID','GramPanchayatID']]

In [ ]:
x = df.values.tolist()

In [ ]:
rgp_id=[]
for i in rv_id:
    for j,k in x:
        if i == j:
            rgp_id.append(k)
for i in pd.read_csv('./grampanchayat.csv')['ID'].values.tolist():
    print(str(i)+" "+str(rgp_id.count(i)),end='---')
rgp_id = sample(rgp_id,len(rgp_id))
rgp_id = sample(rgp_id,len(rgp_id))
rgp_id = sample(rgp_id,len(rgp_id))
len(rgp_id)

In [ ]:
r_id = []
# for i in range(1,tcount+1):
for i in range(1,1+tcount):
    r_id.append(i)
len(r_id)

In [ ]:
father = []
mother = []
while len(father)<name_date_count:
    x=random_names(male,name_date_count)
    father.extend(random_first_name(x))
print(len(father))
while len(mother)<name_date_count:
    x=random_names(female,name_date_count)
    mother.extend(random_first_name(x))
print(len(mother))

In [ ]:
for i in range(len(father)-tcount):
    father.pop()
    i+=1
for i in range(len(mother)-tcount):
    mother.pop()
    i+=1
len(father),len(mother)

In [ ]:
males = []
males_last = []
females = []
females_last = []
while len(males)<name_date_count:
    x = random_names(male,name_date_count)
    males.extend(random_first_name(x))
print('males done'+str(len(males)))

while len(males_last)<name_date_count:
    y = random_names(male,name_date_count)
    males_last.extend(random_last_name(y))
print('males_last done'+str(len(males_last)))

while len(females)<name_date_count:
    x = random_names(female,name_date_count)
    females.extend(random_first_name(x))
print('females done'+str(len(females)))

while len(females_last)<name_date_count:
    y = random_names(female,name_date_count)
    females_last.extend(random_last_name(y))
print("females_last done"+str(len(females_last)))
#------------------------------------------------------------------------------
for i in range(len(males)-tcount):
    males.pop()
print(len(males))
for i in range(len(males_last)-tcount):
    males_last.pop()
print(len(males_last))
for i in range(len(females)-tcount):
    females.pop()
print(len(females))
for i in range(len(females_last)-tcount):
    females_last.pop()
print(len(females_last))

In [ ]:
sex = []
i=0
while i<len(males):
    sex.append('Male')
    i+=1
sex1 = []
i=0
while i<len(females_last):
    sex1.append('Female')
    i+=1

In [ ]:
males = list(zip(males,males_last,sex))
females = list(zip(females,females_last,sex1))

In [ ]:
sex.count('Male')

In [ ]:
name=[]
for i in males:
    name.append(i)
for i in females:
    name.append(i)
name = sample(name,len(name))
name = sample(name,len(name))
name = sample(name,len(name))
for i in range(len(name)-tcount):
    name.pop()
    i+=1
len(name)

In [ ]:
rfirst_name=[]
rlast_name=[]
rsex=[]
for x,y,z in name:
    rfirst_name.append(x)
    rlast_name.append(y)
    rsex.append(z)
len(rfirst_name),len(rlast_name),len(rsex)

In [ ]:
start = 1925
end = date.today().year
rbirth_dates = []
while len(rbirth_dates)<name_date_count:
    rbirth_dates.extend(random_dates(start,end,name_date_count))
    rbirth_dates = sample(rbirth_dates,len(rbirth_dates))
    rbirth_dates = sample(rbirth_dates,len(rbirth_dates))
    print(len(rbirth_dates))
for i in range(len(rbirth_dates)-tcount):
    rbirth_dates.pop()
    i+=1
len(rbirth_dates)

In [ ]:
age = []

dates = pd.to_datetime(rbirth_dates)
for i in range(len(dates)):
    age.append(calculateAge(dates[i]))
len(age)

In [ ]:
for i in range(100):
    print(str(i)+"  "+str(age.count(i)),end=' -> ')

In [ ]:
c = 0
for i in age:
    if i <18:
        c+=1
c

In [ ]:
df = pd.read_csv("./village.csv")[['ID','VillageName']].values.tolist()
birth_place = []
for i in rv_id:
    for j,k in df:
        if i == j:
            birth_place.append(k)
len(birth_place)

In [ ]:
radhaar = []

x = (0,1,2,3,4,5,6,7,8,9,)
i = 0
while i<tcount:
    radhaar.append(str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x)))
    i+=1
len(radhaar)

In [ ]:
var = 'A'
var1 = 'a'
cap_alphabets=[(chr(ord(var)+i)) for i in range(26)]
small_alphabets=[(chr(ord(var1)+i)) for i in range(26)]
num = []
for i in range(9):
    num.append(str(i))
char = []
from string import punctuation
for i in str(punctuation):
    char.append(i)
password = []
for i in cap_alphabets:
    password.append(i)
for j in small_alphabets:
    password.append(j)
for k in num:
    password.append(k)
for l in char:
    password.append(l)
len(password)

In [ ]:
a = [1,2,3,4,5,6,7,8,9,0]
i = 0
login_pass = []
while len(login_pass)<tcount:
    if choice(a)==0:
        login_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==1:
        login_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==2:
        login_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==3:
        login_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==4:
        login_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==5:
        login_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==6:
        login_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==7:
        login_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==8:
        login_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==9:
        login_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
len(login_pass)

# 

In [ ]:
login_pass = []
a = 0
for i in rfirst_name:
    try:
        login_pass.append(i+"_"+str(r_id[a]))
    except IndexError:
        break
    a+=1
len(login_pass)

In [ ]:
rlogin_pass = login_pass

# rlogin_pass = []
for i in login_pass:
    rlogin_pass.append(encrypt_data(i,crypto_key))

In [ ]:
x = random_marital(tcount)
marital = sample(x,len(x))
for i in [1,2,3,4,5]:
    print(marital.count(i),end='\n')
len(marital)

In [ ]:
m = list(zip(age,marital))
marital = []
for i,j in m:
    if i <18:
        j=1
        marital.append(j)
    else:
        marital.append(j)
print(marital.count(1),marital.count(2),marital.count(3),marital.count(4),marital.count(5),len(marital),end='\n')

In [ ]:
x = random_disability(tcount)
disability = sample(x,len(x))
for i in range(1,20):
    print(disability.count(i),end=' ')
len(disability)

In [ ]:
num = [0,1,2,3,4,5,6,7,8,9]
rmobile = []
i = 0
c = 0
while i<tcount:
    i+=1
    if choice(num) in [1,3,4]:
        ph = '9'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    elif choice(num) in [2,5,7]:
        ph = '7'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    elif choice(num) in [6,8,9]:
        ph = '8'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    else:
        c += 1
        ph = '0'+'2'+'2'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    rmobile.append(int(ph))
len(rmobile),c

In [ ]:
n = [0,1,2,3,4,5,6,7,8,9]
remail = []
for i in rfirst_name:
    if choice(n) in [1,2,3,4,6,7,8]:
        mail = i+'@gmail.com'
    elif choice(n) in [0,9]:
        mail = i+'@yahoo.com'
    elif choice(n)==5:
        mail = i+'@outlook.com'
    else:
        mail = i+'@reddit.com'
    remail.append(mail) 
# list(zip(email,first_name))
len(remail)

In [ ]:
title=[]
x = list(zip(age,sex,marital))
for i,j,k in x:
    if i<18 and j == 'Male':
        title.append("S/o")
    elif i>=18 and j == "Male":
        title.append("Mr.")
    elif i <18 and j == "Female":
        title.append("D/o")
    elif i>=18 and j == "Female" and k == 1:
        title.append("Miss")
    else:
        title.append("Mrs.")
len(title)

In [ ]:
add = []
blanktxt = []
blanknum = []
mid_name = []
i = 0
while i<tcount:
    i+=1
    blanktxt.append('')
    blanknum.append(0)
    add.append('')
    mid_name.append("")
len(mid_name)

In [ ]:
radd = []
for i in r_id:
    radd.append('Address')
len(radd)

# 

In [ ]:
list_house = []

In [ ]:
df = pd.read_csv('./village populas.csv',index_col=4)[['populas','house']]
df['p/h'] = df['populas']/df['house']
df['p/h'] = round(df['p/h'])
s = 0
for i in df['house']:
    s+=i
h_id = []
for i in range(1,s+1):
    h_id.append(i)
len(h_id)

In [ ]:
hv_id = []
x = df['house'].values.tolist()
a = 1
for i in x:
    for j in range(i):
        hv_id.append(a)
    a+=1
hv_id = sample(hv_id,len(hv_id))
hv_id = sample(hv_id,len(hv_id))
hv_id = sample(hv_id,len(hv_id))
len(hv_id)

In [ ]:
hgp_id = []
df1 = pd.read_csv("./village.csv")
df1 = df1.drop(df1.columns[2:],axis = 1)
x = df1.values.tolist()
for k in hv_id:
    for i,j in x:
        if i==k:
            hgp_id.append(j)
len(hgp_id)

In [ ]:
h_id = []
for i in range(1,len(hv_id)+1):
    h_id.append(i)
len(h_id)

In [ ]:
# for i in range(1,18):
#     print('print(len(v'+str(i)+'_male_child)',end=',')
#     print('len(v'+str(i)+'_female_child)',end=',')
#     print('len(v'+str(i)+'_male_adult_married)',end=',')
#     print('len(v'+str(i)+'_male_adult_unmarried)',end=',')
#     print('len(v'+str(i)+'_male_adult_married)',end=',')
#     print('len(v'+str(i)+'_male_adult_married)',end=',')
#     print('len(v'+str(i)+'_widowed)',end=',')
#     print('len(v'+str(i)+'_others)',end=')\n')

In [ ]:
for i in range(1,18):
    print("print((len(v"+str(i)+"_male_child)+len(v"+str(i)+"_female_child)+len(v"+str(i)+"_male_adult_unmarried)+len(v"+str(i)+"_female_adult_unmarried)+len(v"+str(i)+"_male_adult_married)+len(v"+str(i)+"_female_adult_married)+len(v"+str(i)+"_widowed)+len(v"+str(i)+"_others)))")
    

In [ ]:
v1_male_child=[]
v1_female_child=[]
v1_male_adult_unmarried=[]
v1_female_adult_unmarried=[]
v1_male_adult_married=[]
v1_female_adult_married=[]
v1_widowed=[]
v1_others=[]

v2_male_child=[]
v2_female_child=[]
v2_male_adult_unmarried=[]
v2_female_adult_unmarried=[]
v2_male_adult_married=[]
v2_female_adult_married=[]
v2_widowed=[]
v2_others=[]

v3_male_child=[]
v3_female_child=[]
v3_male_adult_unmarried=[]
v3_female_adult_unmarried=[]
v3_male_adult_married=[]
v3_female_adult_married=[]
v3_widowed=[]
v3_others=[]

v4_male_child=[]
v4_female_child=[]
v4_male_adult_unmarried=[]
v4_female_adult_unmarried=[]
v4_male_adult_married=[]
v4_female_adult_married=[]
v4_widowed=[]
v4_others=[]

v5_male_child=[]
v5_female_child=[]
v5_male_adult_unmarried=[]
v5_female_adult_unmarried=[]
v5_male_adult_married=[]
v5_female_adult_married=[]
v5_widowed=[]
v5_others=[]

v6_male_child=[]
v6_female_child=[]
v6_male_adult_unmarried=[]
v6_female_adult_unmarried=[]
v6_male_adult_married=[]
v6_female_adult_married=[]
v6_widowed=[]
v6_others=[]

v7_male_child=[]
v7_female_child=[]
v7_male_adult_unmarried=[]
v7_female_adult_unmarried=[]
v7_male_adult_married=[]
v7_female_adult_married=[]
v7_widowed=[]
v7_others=[]

v8_male_child=[]
v8_female_child=[]
v8_male_adult_unmarried=[]
v8_female_adult_unmarried=[]
v8_male_adult_married=[]
v8_female_adult_married=[]
v8_widowed=[]
v8_others=[]

v9_male_child=[]
v9_female_child=[]
v9_male_adult_unmarried=[]
v9_female_adult_unmarried=[]
v9_male_adult_married=[]
v9_female_adult_married=[]
v9_widowed=[]
v9_others=[]

v10_male_child=[]
v10_female_child=[]
v10_male_adult_unmarried=[]
v10_female_adult_unmarried=[]
v10_male_adult_married=[]
v10_female_adult_married=[]
v10_widowed=[]
v10_others=[]

v11_male_child=[]
v11_female_child=[]
v11_male_adult_unmarried=[]
v11_female_adult_unmarried=[]
v11_male_adult_married=[]
v11_female_adult_married=[]
v11_widowed=[]
v11_others=[]

v12_male_child=[]
v12_female_child=[]
v12_male_adult_unmarried=[]
v12_female_adult_unmarried=[]
v12_male_adult_married=[]
v12_female_adult_married=[]
v12_widowed=[]
v12_others=[]

v13_male_child=[]
v13_female_child=[]
v13_male_adult_unmarried=[]
v13_female_adult_unmarried=[]
v13_male_adult_married=[]
v13_female_adult_married=[]
v13_widowed=[]
v13_others=[]

v14_male_child=[]
v14_female_child=[]
v14_male_adult_unmarried=[]
v14_female_adult_unmarried=[]
v14_male_adult_married=[]
v14_female_adult_married=[]
v14_widowed=[]
v14_others=[]

v15_male_child=[]
v15_female_child=[]
v15_male_adult_unmarried=[]
v15_female_adult_unmarried=[]
v15_male_adult_married=[]
v15_female_adult_married=[]
v15_widowed=[]
v15_others=[]

v16_male_child=[]
v16_female_child=[]
v16_male_adult_unmarried=[]
v16_female_adult_unmarried=[]
v16_male_adult_married=[]
v16_female_adult_married=[]
v16_widowed=[]
v16_others=[]

v17_male_child=[]
v17_female_child=[]
v17_male_adult_unmarried=[]
v17_female_adult_unmarried=[]
v17_male_adult_married=[]
v17_female_adult_married=[]
v17_widowed=[]
v17_others=[]

li = list(zip(r_id,rv_id,marital,age,rsex))
for y,x,z,a,b in li:
    if x == 1:
        if a<18 and b=="Male":                                           
            v1_male_child.append((y,x,z,a,b))
        elif a<18 and b=='Female':                                       
            v1_female_child.append((y,x,z,a,b))                      
        elif a>18 and z==2 and b=='Male':                          
            v1_male_adult_married.append((y,x,z,a,b))
        elif a>18 and z==2 and b=='Female':                        
            v1_female_adult_married.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Female':                        
            v1_female_adult_unmarried.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Male':                          
            v1_male_adult_unmarried.append((y,x,z,a,b))
        elif z==3:                                                      
            v1_widowed.append((y,x,z,a,b))
        else:                                                                 
            v1_others.append((y,x,z,a,b))
            
    elif x == 2:
        if a<18 and b=="Male":                                           
            v2_male_child.append((y,x,z,a,b))
        elif a<18 and b=='Female':                                       
            v2_female_child.append((y,x,z,a,b))                      
        elif a>18 and z==2 and b=='Male':                          
            v2_male_adult_married.append((y,x,z,a,b))
        elif a>18 and z==2 and b=='Female':                        
            v2_female_adult_married.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Female':                        
            v2_female_adult_unmarried.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Male':                          
            v2_male_adult_unmarried.append((y,x,z,a,b))
        elif z==3:                                                      
            v2_widowed.append((y,x,z,a,b))
        else:                                                                 
            v2_others.append((y,x,z,a,b))
            
    elif x == 3:
        if a<18 and b=="Male":                                           
            v3_male_child.append((y,x,z,a,b))
        elif a<18 and b=='Female':                                       
            v3_female_child.append((y,x,z,a,b))                      
        elif a>18 and z==2 and b=='Male':                          
            v3_male_adult_married.append((y,x,z,a,b))
        elif a>18 and z==2 and b=='Female':                        
            v3_female_adult_married.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Female':                        
            v3_female_adult_unmarried.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Male':                          
            v3_male_adult_unmarried.append((y,x,z,a,b))
        elif z==3:                                                      
            v3_widowed.append((y,x,z,a,b))
        else:                                                                 
            v3_others.append((y,x,z,a,b))
            
    elif x == 4:
        if a<18 and b=="Male":                                           
            v4_male_child.append((y,x,z,a,b))
        elif a<18 and b=='Female':                                       
            v4_female_child.append((y,x,z,a,b))                      
        elif a>18 and z==2 and b=='Male':                          
            v4_male_adult_married.append((y,x,z,a,b))
        elif a>18 and z==2 and b=='Female':                        
            v4_female_adult_married.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Female':                        
            v4_female_adult_unmarried.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Male':                          
            v4_male_adult_unmarried.append((y,x,z,a,b))
        elif z==3:                                                      
            v4_widowed.append((y,x,z,a,b))
        else:                                                                 
            v4_others.append((y,x,z,a,b))
            
    elif x == 5:
        if a<18 and b=="Male":                                           
            v5_male_child.append((y,x,z,a,b))
        elif a<18 and b=='Female':                                       
            v5_female_child.append((y,x,z,a,b))                      
        elif a>18 and z==2 and b=='Male':                          
            v5_male_adult_married.append((y,x,z,a,b))
        elif a>18 and z==2 and b=='Female':                        
            v5_female_adult_married.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Female':                        
            v5_female_adult_unmarried.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Male':                          
            v5_male_adult_unmarried.append((y,x,z,a,b))
        elif z==3:                                                      
            v5_widowed.append((y,x,z,a,b))
        else:                                                                 
            v5_others.append((y,x,z,a,b))
            
    elif x == 6:
        if a<18 and b=="Male":                                           
            v6_male_child.append((y,x,z,a,b))
        elif a<18 and b=='Female':                                       
            v6_female_child.append((y,x,z,a,b))                      
        elif a>18 and z==2 and b=='Male':                          
            v6_male_adult_married.append((y,x,z,a,b))
        elif a>18 and z==2 and b=='Female':                        
            v6_female_adult_married.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Female':                        
            v6_female_adult_unmarried.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Male':                          
            v6_male_adult_unmarried.append((y,x,z,a,b))
        elif z==3:                                                      
            v6_widowed.append((y,x,z,a,b))
        else:                                                                 
            v6_others.append((y,x,z,a,b))
    
    elif x == 7:
        if a<18 and b=="Male":                                           
            v7_male_child.append((y,x,z,a,b))
        elif a<18 and b=='Female':                                       
            v7_female_child.append((y,x,z,a,b))                      
        elif a>18 and z==2 and b=='Male':                          
            v7_male_adult_married.append((y,x,z,a,b))
        elif a>18 and z==2 and b=='Female':                        
            v7_female_adult_married.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Female':                        
            v7_female_adult_unmarried.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Male':                          
            v7_male_adult_unmarried.append((y,x,z,a,b))
        elif z==3:                                                      
            v7_widowed.append((y,x,z,a,b))
        else:                                                                 
            v7_others.append((y,x,z,a,b))
    
    elif x == 8:
        if a<18 and b=="Male":                                           
            v8_male_child.append((y,x,z,a,b))
        elif a<18 and b=='Female':                                       
            v8_female_child.append((y,x,z,a,b))                      
        elif a>18 and z==2 and b=='Male':                          
            v8_male_adult_married.append((y,x,z,a,b))
        elif a>18 and z==2 and b=='Female':                        
            v8_female_adult_married.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Female':                        
            v8_female_adult_unmarried.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Male':                          
            v8_male_adult_unmarried.append((y,x,z,a,b))
        elif z==3:                                                      
            v8_widowed.append((y,x,z,a,b))
        else:                                                                 
            v8_others.append((y,x,z,a,b))
    
    elif x == 9:
        if a<18 and b=="Male":                                           
            v9_male_child.append((y,x,z,a,b))
        elif a<18 and b=='Female':                                       
            v9_female_child.append((y,x,z,a,b))                      
        elif a>18 and z==2 and b=='Male':                          
            v9_male_adult_married.append((y,x,z,a,b))
        elif a>18 and z==2 and b=='Female':                        
            v9_female_adult_married.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Female':                        
            v9_female_adult_unmarried.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Male':                          
            v9_male_adult_unmarried.append((y,x,z,a,b))
        elif z==3:                                                      
            v9_widowed.append((y,x,z,a,b))
        else:                                                                 
            v9_others.append((y,x,z,a,b))
    
    elif x == 10:
        if a<18 and b=="Male":                                           
            v10_male_child.append((y,x,z,a,b))
        elif a<18 and b=='Female':                                       
            v10_female_child.append((y,x,z,a,b))                      
        elif a>18 and z==2 and b=='Male':                          
            v10_male_adult_married.append((y,x,z,a,b))
        elif a>18 and z==2 and b=='Female':                        
            v10_female_adult_married.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Female':                        
            v10_female_adult_unmarried.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Male':                          
            v10_male_adult_unmarried.append((y,x,z,a,b))
        elif z==3:                                                      
            v10_widowed.append((y,x,z,a,b))
        else:                                                                 
            v10_others.append((y,x,z,a,b))
   
    elif x == 11:
        if a<18 and b=="Male":                                           
            v11_male_child.append((y,x,z,a,b))
        elif a<18 and b=='Female':                                       
            v11_female_child.append((y,x,z,a,b))                      
        elif a>18 and z==2 and b=='Male':                          
            v11_male_adult_married.append((y,x,z,a,b))
        elif a>18 and z==2 and b=='Female':                        
            v11_female_adult_married.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Female':                        
            v11_female_adult_unmarried.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Male':                          
            v11_male_adult_unmarried.append((y,x,z,a,b))
        elif z==3:                                                      
            v11_widowed.append((y,x,z,a,b))
        else:                                                                 
            v11_others.append((y,x,z,a,b))
    
    elif x == 12:
        if a<18 and b=="Male":                                           
            v12_male_child.append((y,x,z,a,b))
        elif a<18 and b=='Female':                                       
            v12_female_child.append((y,x,z,a,b))                      
        elif a>18 and z==2 and b=='Male':                          
            v12_male_adult_married.append((y,x,z,a,b))
        elif a>18 and z==2 and b=='Female':                        
            v12_female_adult_married.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Female':                        
            v12_female_adult_unmarried.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Male':                          
            v12_male_adult_unmarried.append((y,x,z,a,b))
        elif z==3:                                                      
            v12_widowed.append((y,x,z,a,b))
        else:                                                                 
            v12_others.append((y,x,z,a,b))
    
    elif x == 13:
        if a<18 and b=="Male":                                           
            v13_male_child.append((y,x,z,a,b))
        elif a<18 and b=='Female':                                       
            v13_female_child.append((y,x,z,a,b))                      
        elif a>18 and z==2 and b=='Male':                          
            v13_male_adult_married.append((y,x,z,a,b))
        elif a>18 and z==2 and b=='Female':                        
            v13_female_adult_married.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Female':                        
            v13_female_adult_unmarried.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Male':                          
            v13_male_adult_unmarried.append((y,x,z,a,b))
        elif z==3:                                                      
            v13_widowed.append((y,x,z,a,b))
        else:                                                                 
            v13_others.append((y,x,z,a,b))
    
    elif x == 14:
        if a<18 and b=="Male":                                           
            v14_male_child.append((y,x,z,a,b))
        elif a<18 and b=='Female':                                       
            v14_female_child.append((y,x,z,a,b))                      
        elif a>18 and z==2 and b=='Male':                          
            v14_male_adult_married.append((y,x,z,a,b))
        elif a>18 and z==2 and b=='Female':                        
            v14_female_adult_married.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Female':                        
            v14_female_adult_unmarried.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Male':                          
            v14_male_adult_unmarried.append((y,x,z,a,b))
        elif z==3:                                                      
            v14_widowed.append((y,x,z,a,b))
        else:                                                                 
            v14_others.append((y,x,z,a,b))
    
    elif x == 15:
        if a<18 and b=="Male":                                           
            v15_male_child.append((y,x,z,a,b))
        elif a<18 and b=='Female':                                       
            v15_female_child.append((y,x,z,a,b))                      
        elif a>18 and z==2 and b=='Male':                          
            v15_male_adult_married.append((y,x,z,a,b))
        elif a>18 and z==2 and b=='Female':                        
            v15_female_adult_married.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Female':                        
            v15_female_adult_unmarried.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Male':                          
            v15_male_adult_unmarried.append((y,x,z,a,b))
        elif z==3:                                                      
            v15_widowed.append((y,x,z,a,b))
        else:                                                                 
            v15_others.append((y,x,z,a,b))
    
    elif x == 16:
        if a<18 and b=="Male":                                           
            v16_male_child.append((y,x,z,a,b))
        elif a<18 and b=='Female':                                       
            v16_female_child.append((y,x,z,a,b))                      
        elif a>18 and z==2 and b=='Male':                          
            v16_male_adult_married.append((y,x,z,a,b))
        elif a>18 and z==2 and b=='Female':                        
            v16_female_adult_married.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Female':                        
            v16_female_adult_unmarried.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Male':                          
            v16_male_adult_unmarried.append((y,x,z,a,b))
        elif z==3:                                                      
            v16_widowed.append((y,x,z,a,b))
        else:                                                                 
            v16_others.append((y,x,z,a,b))
    
    elif x == 17:
        if a<18 and b=="Male":                                           
            v17_male_child.append((y,x,z,a,b))
        elif a<18 and b=='Female':                                       
            v17_female_child.append((y,x,z,a,b))                      
        elif a>18 and z==2 and b=='Male':                          
            v17_male_adult_married.append((y,x,z,a,b))
        elif a>18 and z==2 and b=='Female':                        
            v17_female_adult_married.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Female':                        
            v17_female_adult_unmarried.append((y,x,z,a,b))
        elif a>18 and z==1 and b=='Male':                          
            v17_male_adult_unmarried.append((y,x,z,a,b))
        elif z==3:                                                      
            v17_widowed.append((y,x,z,a,b))
        else:                                                                 
            v17_others.append((y,x,z,a,b))

print(len(v1_male_child),len(v1_female_child),len(v1_male_adult_married),len(v1_male_adult_unmarried),len(v1_male_adult_married),len(v1_male_adult_married),len(v1_widowed),len(v1_others))
print(len(v2_male_child),len(v2_female_child),len(v2_male_adult_married),len(v2_male_adult_unmarried),len(v2_male_adult_married),len(v2_male_adult_married),len(v2_widowed),len(v2_others))
print(len(v3_male_child),len(v3_female_child),len(v3_male_adult_married),len(v3_male_adult_unmarried),len(v3_male_adult_married),len(v3_male_adult_married),len(v3_widowed),len(v3_others))
print(len(v4_male_child),len(v4_female_child),len(v4_male_adult_married),len(v4_male_adult_unmarried),len(v4_male_adult_married),len(v4_male_adult_married),len(v4_widowed),len(v4_others))
print(len(v5_male_child),len(v5_female_child),len(v5_male_adult_married),len(v5_male_adult_unmarried),len(v5_male_adult_married),len(v5_male_adult_married),len(v5_widowed),len(v5_others))
print(len(v6_male_child),len(v6_female_child),len(v6_male_adult_married),len(v6_male_adult_unmarried),len(v6_male_adult_married),len(v6_male_adult_married),len(v6_widowed),len(v6_others))
print(len(v7_male_child),len(v7_female_child),len(v7_male_adult_married),len(v7_male_adult_unmarried),len(v7_male_adult_married),len(v7_male_adult_married),len(v7_widowed),len(v7_others))
print(len(v8_male_child),len(v8_female_child),len(v8_male_adult_married),len(v8_male_adult_unmarried),len(v8_male_adult_married),len(v8_male_adult_married),len(v8_widowed),len(v8_others))
print(len(v9_male_child),len(v9_female_child),len(v9_male_adult_married),len(v9_male_adult_unmarried),len(v9_male_adult_married),len(v9_male_adult_married),len(v9_widowed),len(v9_others))
print(len(v10_male_child),len(v10_female_child),len(v10_male_adult_married),len(v10_male_adult_unmarried),len(v10_male_adult_married),len(v10_male_adult_married),len(v10_widowed),len(v10_others))
print(len(v11_male_child),len(v11_female_child),len(v11_male_adult_married),len(v11_male_adult_unmarried),len(v11_male_adult_married),len(v11_male_adult_married),len(v11_widowed),len(v11_others))
print(len(v12_male_child),len(v12_female_child),len(v12_male_adult_married),len(v12_male_adult_unmarried),len(v12_male_adult_married),len(v12_male_adult_married),len(v12_widowed),len(v12_others))
print(len(v13_male_child),len(v13_female_child),len(v13_male_adult_married),len(v13_male_adult_unmarried),len(v13_male_adult_married),len(v13_male_adult_married),len(v13_widowed),len(v13_others))
print(len(v14_male_child),len(v14_female_child),len(v14_male_adult_married),len(v14_male_adult_unmarried),len(v14_male_adult_married),len(v14_male_adult_married),len(v14_widowed),len(v14_others))
print(len(v15_male_child),len(v15_female_child),len(v15_male_adult_married),len(v15_male_adult_unmarried),len(v15_male_adult_married),len(v15_male_adult_married),len(v15_widowed),len(v15_others))
print(len(v16_male_child),len(v16_female_child),len(v16_male_adult_married),len(v16_male_adult_unmarried),len(v16_male_adult_married),len(v16_male_adult_married),len(v16_widowed),len(v16_others))
print(len(v17_male_child),len(v17_female_child),len(v17_male_adult_married),len(v17_male_adult_unmarried),len(v17_male_adult_married),len(v17_male_adult_married),len(v17_widowed),len(v17_others))

In [ ]:
li = list(zip(h_id,hv_id))
v1=[]
v2=[]
v3=[]
v4=[]
v5=[]
v6=[]
v7=[]
v8=[]
v9=[]
v10=[]
v11=[]
v12=[]
v13=[]
v14=[]
v15=[]
v16=[]
v17=[]
for x,y in li:
    if y == 1:
        v1.append(x)
    if y == 2:
        v2.append(x)
    if y == 3:
        v3.append(x)
    if y == 4:
        v4.append(x)
    if y == 5:
        v5.append(x)
    if y == 6:
        v6.append(x)
    if y == 7:
        v7.append(x)
    if y == 8:
        v8.append(x)
    if y == 9:
        v9.append(x)
    if y == 10:
        v10.append(x)
    if y == 11:
        v11.append(x)
    if y == 12:
        v12.append(x)
    if y == 13:
        v13.append(x)
    if y == 14:
        v14.append(x)
    if y == 15:
        v15.append(x)
    if y == 16:
        v16.append(x)
    if y == 17:
        v17.append(x)
    
len(v1),len(v2),len(v3),len(v4),len(v5),len(v6),len(v7),len(v8),len(v9),len(v10),len(v11),len(v12),len(v13),len(v14),len(v15),len(v16),len(v17)

In [ ]:
house_res_id =[]
final = []

In [ ]:
# %time
                                                    ########## Village1 ################
boy = sample(v1_male_child,len(v1_male_child))
girl = sample(v1_female_child,len(v1_female_child))
husband = sample(v1_male_adult_married,len(v1_male_adult_married))
wife = sample(v1_female_adult_married,len(v1_female_adult_married))
man = sample(v1_male_adult_unmarried,len(v1_male_adult_unmarried))
woman = sample(v1_female_adult_unmarried,len(v1_female_adult_unmarried))
widowed = sample(v1_widowed,len(v1_widowed))
others = sample(v1_others,len(v1_others))
house = sample(v1,len(v1))

In [ ]:
h1 = []
count = 1
while len(house)!=0:
    print(count)
    count+=1
    try:
        for i in range(len(house)):                                      
            h = house.pop(randrange(len(house)))
            ch = choice(range(10))
            if ch in [1,2]:                                              #3
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                            else:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                    else:
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g = list(woman.pop(randrange(len(woman))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(man.pop(randrange(len(man))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                else:
                    w = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w
                    w.append(h)
                    w.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
            elif ch in [3,4,5]:                                                     #5
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    o = list(others.pop(randrange(len(others))))
                    o.append(h)
                    o.append(0)
                    o.append(0)
                    o.append(0)
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,g,b,g1))
                            else:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,g,b,b1))
                        elif choice(range(1))==1:
                            if len(man)!=0:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w1,w,g,b,b1))
                            else:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w1,w,g,b,g1))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))
                            else:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                            else:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))                        
            else:                                                 #4
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,b,g))
    except ValueError:
        print(h)
        h1.append(h)
        pass
while len(h1)!=0:
    h = h1.pop(randrange(len(h1)))
    try:
        w = list(widowed.pop(randrange(len(widowed))))
    except ValueError:
        h1.append(h)
        print('last done')
        print(h)
        break
    x,y,z,a,b = w
    w.append(h)
    w.append(0)
    w.append(0)
    w.append(0)
    house_res_id.append((h,x))
    if a>45:
        try:
            g = list(woman.pop(randrange(len(woman))))
        except ValueError:
            try:
                g = list(man.pop(randrange(len(man))))
            except ValueError:
                g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(man.pop(randrange(len(man))))
        except ValueError:
            b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))
    else:
        try:
            g = list(girl.pop(randrange(len(girl))))
        except ValueError:
            g = list(boy.pop(randrange(len(boy))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))

In [ ]:
a = 1
t = 0
for i in [boy,girl,man,woman,wife,husband,widowed,others]:
    r=0
    if len(i)!=0:
        for j in i:
            if len(h1)!=0:
                h = h1.pop(randrange(len((h1))))
                t+=1
                r+=1
                j = list(j)
                j.append(h)
                j.append(0)
                final.append(j)
                house_res_id.append((h,j[0]))
            else:
                t+=1
                r+=1
                j = list(j)
                j.append(0)
                j.append(0)
                final.append(j)
        print(a,r,t)
        a+=1
    else:
        pass

In [ ]:
# %time
                                                    ########## Village2 ################
boy = sample(v2_male_child,len(v2_male_child))
girl = sample(v2_female_child,len(v2_female_child))
husband = sample(v2_male_adult_married,len(v2_male_adult_married))
wife = sample(v2_female_adult_married,len(v2_female_adult_married))
man = sample(v2_male_adult_unmarried,len(v2_male_adult_unmarried))
woman = sample(v2_female_adult_unmarried,len(v2_female_adult_unmarried))
widowed = sample(v2_widowed,len(v2_widowed))
others = sample(v2_others,len(v2_others))
house = sample(v2,len(v2))

In [ ]:
h1 = []
count = 1
while len(house)!=0:
    print(count)
    count+=1
    try:
        for i in range(len(house)):                                      
            h = house.pop(randrange(len(house)))
            ch = choice(range(10))
            if ch in [1,2]:                                              #3
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                            else:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                    else:
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g = list(woman.pop(randrange(len(woman))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(man.pop(randrange(len(man))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                else:
                    w = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w
                    w.append(h)
                    w.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
            elif ch in [3,4,5]:                                                     #5
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    o = list(others.pop(randrange(len(others))))
                    o.append(h)
                    o.append(0)
                    o.append(0)
                    o.append(0)
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,g,b,g1))
                            else:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,g,b,b1))
                        elif choice(range(1))==1:
                            if len(man)!=0:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w1,w,g,b,b1))
                            else:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w1,w,g,b,g1))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))
                            else:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                            else:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))                        
            else:                                                 #4
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,b,g))
    except ValueError:
        print(h)
        h1.append(h)
        pass
while len(h1)!=0:
    h = h1.pop(randrange(len(h1)))
    try:
        w = list(widowed.pop(randrange(len(widowed))))
    except ValueError:
        h1.append(h)
        print('last done')
        print(h)
        break
    x,y,z,a,b = w
    w.append(h)
    w.append(0)
    w.append(0)
    w.append(0)
    house_res_id.append((h,x))
    if a>45:
        try:
            g = list(woman.pop(randrange(len(woman))))
        except ValueError:
            try:
                g = list(man.pop(randrange(len(man))))
            except ValueError:
                g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(man.pop(randrange(len(man))))
        except ValueError:
            b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))
    else:
        g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))

In [ ]:
a = 1
t = 0
for i in [boy,girl,man,woman,wife,husband,widowed,others]:
    r=0
    if len(i)!=0:
        for j in i:
            if len(h1)!=0:
                h = h1.pop(randrange(len((h1))))
                t+=1
                r+=1
                j = list(j)
                j.append(h)
                j.append(0)
                final.append(j)
                house_res_id.append((h,j[0]))
            else:
                t+=1
                r+=1
                j = list(j)
                j.append(0)
                j.append(0)
                final.append(j)
        print(a,r,t)
        a+=1
    else:
        pass

In [ ]:
# %time
                                                    ########## Village3 ################
boy = sample(v3_male_child,len(v3_male_child))
girl = sample(v3_female_child,len(v3_female_child))
husband = sample(v3_male_adult_married,len(v3_male_adult_married))
wife = sample(v3_female_adult_married,len(v3_female_adult_married))
man = sample(v3_male_adult_unmarried,len(v3_male_adult_unmarried))
woman = sample(v3_female_adult_unmarried,len(v3_female_adult_unmarried))
widowed = sample(v3_widowed,len(v3_widowed))
others = sample(v3_others,len(v3_others))
house = sample(v3,len(v3))

In [ ]:
h1 = []
count = 1
while len(house)!=0:
    print(count)
    count+=1
    try:
        for i in range(len(house)):                                      
            h = house.pop(randrange(len(house)))
            ch = choice(range(10))
            if ch in [1,2]:                                              #3
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                            else:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                    else:
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g = list(woman.pop(randrange(len(woman))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(man.pop(randrange(len(man))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                else:
                    w = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w
                    w.append(h)
                    w.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
            elif ch in [3,4,5]:                                                     #5
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    o = list(others.pop(randrange(len(others))))
                    o.append(h)
                    o.append(0)
                    o.append(0)
                    o.append(0)
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,g,b,g1))
                            else:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,g,b,b1))
                        elif choice(range(1))==1:
                            if len(man)!=0:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w1,w,g,b,b1))
                            else:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w1,w,g,b,g1))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))
                            else:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                            else:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))                        
            else:                                                 #4
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,b,g))
    except ValueError:
        print(h)
        h1.append(h)
        pass
while len(h1)!=0:
    h = h1.pop(randrange(len(h1)))
    try:
        w = list(widowed.pop(randrange(len(widowed))))
    except ValueError:
        h1.append(h)
        print('last done')
        print(h)
        break
    x,y,z,a,b = w
    w.append(h)
    w.append(0)
    w.append(0)
    w.append(0)
    house_res_id.append((h,x))
    if a>45:
        try:
            g = list(woman.pop(randrange(len(woman))))
        except ValueError:
            try:
                g = list(man.pop(randrange(len(man))))
            except ValueError:
                g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(man.pop(randrange(len(man))))
        except ValueError:
            b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))
    else:
        try:
            g = list(girl.pop(randrange(len(girl))))
        except ValueError:
            try:
                g = list(boy.pop(randrange(len(boy))))
            except ValueError:
                g = list(woman.pop(randrange(len(woman))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(boy.pop(randrange(len(boy))))
        except ValueError:
            b = list(man.pop(randrange(len(man))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))

In [ ]:
len(boy),len(girl),len(man),len(woman),len(wife),len(husband),len(widowed),len(others),len(house)

In [ ]:
a = 1
t = 0
for i in [boy,girl,man,woman,wife,husband,widowed,others]:
    r=0
    if len(i)!=0:
        for j in i:
            if len(h1)!=0:
                h = h1.pop(randrange(len((h1))))
                t+=1
                r+=1
                j = list(j)
                j.append(h)
                j.append(0)
                final.append(j)
                house_res_id.append((h,j[0]))
            else:
                t+=1
                r+=1
                j = list(j)
                j.append(0)
                j.append(0)
                final.append(j)
        print(a,r,t)
        a+=1
    else:
        pass

In [ ]:
# %time
                                                    ########## Village4 ################
boy = sample(v4_male_child,len(v4_male_child))
girl = sample(v4_female_child,len(v4_female_child))
husband = sample(v4_male_adult_married,len(v4_male_adult_married))
wife = sample(v4_female_adult_married,len(v4_female_adult_married))
man = sample(v4_male_adult_unmarried,len(v4_male_adult_unmarried))
woman = sample(v4_female_adult_unmarried,len(v4_female_adult_unmarried))
widowed = sample(v4_widowed,len(v4_widowed))
others = sample(v4_others,len(v4_others))
house = sample(v4,len(v4))

In [ ]:
h1 = []
count = 1
while len(house)!=0:
    print(count)
    count+=1
    try:
        for i in range(len(house)):                                      
            h = house.pop(randrange(len(house)))
            ch = choice(range(10))
            if ch in [1,2]:                                              #3
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                            else:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                    else:
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g = list(woman.pop(randrange(len(woman))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(man.pop(randrange(len(man))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                else:
                    w = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w
                    w.append(h)
                    w.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
            elif ch in [3,4,5]:                                                     #5
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    o = list(others.pop(randrange(len(others))))
                    o.append(h)
                    o.append(0)
                    o.append(0)
                    o.append(0)
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,g,b,g1))
                            else:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,g,b,b1))
                        elif choice(range(1))==1:
                            if len(man)!=0:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w1,w,g,b,b1))
                            else:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w1,w,g,b,g1))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))
                            else:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                            else:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))                        
            else:                                                 #4
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,b,g))
    except ValueError:
        print(h)
        h1.append(h)
        pass
while len(h1)!=0:
    h = h1.pop(randrange(len(h1)))
    try:
        w = list(widowed.pop(randrange(len(widowed))))
    except ValueError:
        h1.append(h)
        print('last done')
        print(h)
        break
    x,y,z,a,b = w
    w.append(h)
    w.append(0)
    w.append(0)
    w.append(0)
    house_res_id.append((h,x))
    if a>45:
        try:
            g = list(woman.pop(randrange(len(woman))))
        except ValueError:
            try:
                g = list(man.pop(randrange(len(man))))
            except ValueError:
                g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(man.pop(randrange(len(man))))
        except ValueError:
            b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))
    else:
        try:
            g = list(girl.pop(randrange(len(girl))))
        except ValueError:
            try:
                g = list(boy.pop(randrange(len(boy))))
            except ValueError:
                try:
                    g = list(woman.pop(randrange(len(woman))))
                except ValueError:
                    g = list(man.pop(randrange(len(man))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(boy.pop(randrange(len(boy))))
        except ValueError:
            try:
                b = list(girl.pop(randrange(len(girl))))
            except ValueError:
                try:
                    b = list(man.pop(randrange(len(girl))))
                except ValueError:
                    b = list(woman.pop(randrange(len(woman))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))

In [ ]:
a = 1
t = 0
for i in [boy,girl,man,woman,wife,husband,widowed,others]:
    r=0
    if len(i)!=0:
        for j in i:
            if len(h1)!=0:
                h = h1.pop(randrange(len((h1))))
                t+=1
                r+=1
                j = list(j)
                j.append(h)
                j.append(0)
                final.append(j)
                house_res_id.append((h,j[0]))
            else:
                t+=1
                r+=1
                j = list(j)
                j.append(0)
                j.append(0)
                final.append(j)
        print(a,r,t)
        a+=1
    else:
        pass

In [ ]:
# %time
                                                    ########## Village5 ################
boy = sample(v5_male_child,len(v5_male_child))
girl = sample(v5_female_child,len(v5_female_child))
husband = sample(v5_male_adult_married,len(v5_male_adult_married))
wife = sample(v5_female_adult_married,len(v5_female_adult_married))
man = sample(v5_male_adult_unmarried,len(v5_male_adult_unmarried))
woman = sample(v5_female_adult_unmarried,len(v5_female_adult_unmarried))
widowed = sample(v5_widowed,len(v5_widowed))
others = sample(v5_others,len(v5_others))
house = sample(v5,len(v5))

In [ ]:
h1 = []
count = 1
while len(house)!=0:
    print(count)
    count+=1
    try:
        for i in range(len(house)):                                      
            h = house.pop(randrange(len(house)))
            ch = choice(range(10))
            if ch in [1,2]:                                              #3
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                            else:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                    else:
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g = list(woman.pop(randrange(len(woman))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(man.pop(randrange(len(man))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                else:
                    w = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w
                    w.append(h)
                    w.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
            elif ch in [3,4,5]:                                                     #5
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    o = list(others.pop(randrange(len(others))))
                    o.append(h)
                    o.append(0)
                    o.append(0)
                    o.append(0)
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,g,b,g1))
                            else:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,g,b,b1))
                        elif choice(range(1))==1:
                            if len(man)!=0:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w1,w,g,b,b1))
                            else:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w1,w,g,b,g1))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))
                            else:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                            else:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))                        
            else:                                                 #4
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,b,g))
    except ValueError:
        print(h)
        h1.append(h)
        pass
while len(h1)!=0:
    h = h1.pop(randrange(len(h1)))
    try:
        w = list(widowed.pop(randrange(len(widowed))))
    except ValueError:
        h1.append(h)
        print('last done')
        print(h)
        break
    x,y,z,a,b = w
    w.append(h)
    w.append(0)
    w.append(0)
    w.append(0)
    house_res_id.append((h,x))
    if a>45:
        try:
            g = list(woman.pop(randrange(len(woman))))
        except ValueError:
            try:
                g = list(man.pop(randrange(len(man))))
            except ValueError:
                g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(man.pop(randrange(len(man))))
        except ValueError:
            b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))
    else:
        try:
            g = list(girl.pop(randrange(len(girl))))
        except ValueError:
            g = list(boy.pop(randrange(len(boy))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(boy.pop(randrange(len(boy))))
        except ValueError:
            b = list(man.pop(randrange(len(man))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))

In [ ]:
a = 1
t = 0
for i in [boy,girl,man,woman,wife,husband,widowed,others]:
    r=0
    if len(i)!=0:
        for j in i:
            if len(h1)!=0:
                h = h1.pop(randrange(len((h1))))
                t+=1
                r+=1
                j = list(j)
                j.append(h)
                j.append(0)
                final.append(j)
                house_res_id.append((h,j[0]))
            else:
                t+=1
                r+=1
                j = list(j)
                j.append(0)
                j.append(0)
                final.append(j)
        print(a,r,t)
        a+=1
    else:
        pass

In [ ]:
len(boy),len(girl),len(man),len(woman),len(wife),len(husband),len(widowed),len(others),len(house)

In [ ]:
# %time
                                                    ########## Village6 ################
boy = sample(v6_male_child,len(v6_male_child))
girl = sample(v6_female_child,len(v6_female_child))
husband = sample(v6_male_adult_married,len(v6_male_adult_married))
wife = sample(v6_female_adult_married,len(v6_female_adult_married))
man = sample(v6_male_adult_unmarried,len(v6_male_adult_unmarried))
woman = sample(v6_female_adult_unmarried,len(v6_female_adult_unmarried))
widowed = sample(v6_widowed,len(v6_widowed))
others = sample(v6_others,len(v6_others))
house = sample(v6,len(v6))

In [ ]:
h1 = []
count = 1
while len(house)!=0:
    print(count)
    count+=1
    try:
        for i in range(len(house)):                                      
            h = house.pop(randrange(len(house)))
            ch = choice(range(10))
            if ch in [1,2]:                                              #3
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                            else:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                    else:
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g = list(woman.pop(randrange(len(woman))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(man.pop(randrange(len(man))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                else:
                    w = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w
                    w.append(h)
                    w.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
            elif ch in [3,4,5]:                                                     #5
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    o = list(others.pop(randrange(len(others))))
                    o.append(h)
                    o.append(0)
                    o.append(0)
                    o.append(0)
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,g,b,g1))
                            else:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,g,b,b1))
                        elif choice(range(1))==1:
                            if len(man)!=0:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w1,w,g,b,b1))
                            else:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w1,w,g,b,g1))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))
                            else:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                            else:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))                        
            else:                                                 #4
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,b,g))
    except ValueError:
        print(h)
        h1.append(h)
        pass
while len(h1)!=0:
    h = h1.pop(randrange(len(h1)))
    try:
        w = list(widowed.pop(randrange(len(widowed))))
    except ValueError:
        h1.append(h)
        print('last done')
        print(h)
        break
    x,y,z,a,b = w
    w.append(h)
    w.append(0)
    w.append(0)
    w.append(0)
    house_res_id.append((h,x))
    if a>45:
        try:
            g = list(woman.pop(randrange(len(woman))))
        except ValueError:
            try:
                g = list(man.pop(randrange(len(man))))
            except ValueError:
                g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(man.pop(randrange(len(man))))
        except ValueError:
            b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))
    else:
        try:
            g = list(girl.pop(randrange(len(girl))))
        except ValueError:
            g = list(boy.pop(randrange(len(boy))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))

In [ ]:
a = 1
t = 0
for i in [boy,girl,man,woman,wife,husband,widowed,others]:
    r=0
    if len(i)!=0:
        for j in i:
            if len(h1)!=0:
                h = h1.pop(randrange(len((h1))))
                t+=1
                r+=1
                j = list(j)
                j.append(h)
                j.append(0)
                final.append(j)
                house_res_id.append((h,j[0]))
            else:
                t+=1
                r+=1
                j = list(j)
                j.append(0)
                j.append(0)
                final.append(j)
        print(a,r,t)
        a+=1
    else:
        pass

In [ ]:
len(boy),len(girl),len(man),len(woman),len(wife),len(husband),len(widowed),len(others),len(house)

In [ ]:
# %time
                                                    ########## Village7 ################
boy = sample(v7_male_child,len(v7_male_child))
girl = sample(v7_female_child,len(v7_female_child))
husband = sample(v7_male_adult_married,len(v7_male_adult_married))
wife = sample(v7_female_adult_married,len(v7_female_adult_married))
man = sample(v7_male_adult_unmarried,len(v7_male_adult_unmarried))
woman = sample(v7_female_adult_unmarried,len(v7_female_adult_unmarried))
widowed = sample(v7_widowed,len(v7_widowed))
others = sample(v7_others,len(v7_others))
house = sample(v7,len(v7))

In [ ]:
h1 = []
count = 1
while len(house)!=0:
    print(count)
    count+=1
    try:
        for i in range(len(house)):                                      
            h = house.pop(randrange(len(house)))
            ch = choice(range(10))
            if ch in [1,2]:                                              #3
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                            else:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                    else:
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g = list(woman.pop(randrange(len(woman))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(man.pop(randrange(len(man))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                else:
                    w = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w
                    w.append(h)
                    w.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
            elif ch in [3,4,5]:                                                     #5
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    o = list(others.pop(randrange(len(others))))
                    o.append(h)
                    o.append(0)
                    o.append(0)
                    o.append(0)
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,g,b,g1))
                            else:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,g,b,b1))
                        elif choice(range(1))==1:
                            if len(man)!=0:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w1,w,g,b,b1))
                            else:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w1,w,g,b,g1))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))
                            else:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                            else:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))                        
            else:                                                 #4
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,b,g))
    except ValueError:
        print(h)
        h1.append(h)
        pass
while len(h1)!=0:
    h = h1.pop(randrange(len(h1)))
    try:
        w = list(widowed.pop(randrange(len(widowed))))
    except ValueError:
        h1.append(h)
        print('last done')
        print(h)
        break
    x,y,z,a,b = w
    w.append(h)
    w.append(0)
    w.append(0)
    w.append(0)
    house_res_id.append((h,x))
    if a>45:
        try:
            g = list(woman.pop(randrange(len(woman))))
        except ValueError:
            try:
                g = list(man.pop(randrange(len(man))))
            except ValueError:
                g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(man.pop(randrange(len(man))))
        except ValueError:
            b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))
    else:
        g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))

In [ ]:
a = 1
t = 0
for i in [boy,girl,man,woman,wife,husband,widowed,others]:
    r=0
    if len(i)!=0:
        for j in i:
            if len(h1)!=0:
                h = h1.pop(randrange(len((h1))))
                t+=1
                r+=1
                j = list(j)
                j.append(h)
                j.append(0)
                final.append(j)
                house_res_id.append((h,j[0]))
            else:
                t+=1
                r+=1
                j = list(j)
                j.append(0)
                j.append(0)
                final.append(j)
        print(a,r,t)
        a+=1
    else:
        pass

In [ ]:
# %time
                                                    ########## Village8 ################
boy = sample(v8_male_child,len(v8_male_child))
girl = sample(v8_female_child,len(v8_female_child))
husband = sample(v8_male_adult_married,len(v8_male_adult_married))
wife = sample(v8_female_adult_married,len(v8_female_adult_married))
man = sample(v8_male_adult_unmarried,len(v8_male_adult_unmarried))
woman = sample(v8_female_adult_unmarried,len(v8_female_adult_unmarried))
widowed = sample(v8_widowed,len(v8_widowed))
others = sample(v8_others,len(v8_others))
house = sample(v8,len(v8))

In [ ]:
h1 = []
count = 1
while len(house)!=0:
    print(count)
    count+=1
    try:
        for i in range(len(house)):                                      
            h = house.pop(randrange(len(house)))
            ch = choice(range(10))
            if ch in [1,2]:                                              #3
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                            else:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                    else:
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g = list(woman.pop(randrange(len(woman))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(man.pop(randrange(len(man))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                else:
                    w = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w
                    w.append(h)
                    w.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
            elif ch in [3,4,5]:                                                     #5
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    o = list(others.pop(randrange(len(others))))
                    o.append(h)
                    o.append(0)
                    o.append(0)
                    o.append(0)
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,g,b,g1))
                            else:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,g,b,b1))
                        elif choice(range(1))==1:
                            if len(man)!=0:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w1,w,g,b,b1))
                            else:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w1,w,g,b,g1))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))
                            else:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                            else:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))                        
            else:                                                 #4
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,b,g))
    except ValueError:
        print(h)
        h1.append(h)
        pass
while len(h1)!=0:
    h = h1.pop(randrange(len(h1)))
    try:
        w = list(widowed.pop(randrange(len(widowed))))
    except ValueError:
        h1.append(h)
        print('last done')
        print(h)
        break
    x,y,z,a,b = w
    w.append(h)
    w.append(0)
    w.append(0)
    w.append(0)
    house_res_id.append((h,x))
    if a>45:
        try:
            g = list(woman.pop(randrange(len(woman))))
        except ValueError:
            try:
                g = list(man.pop(randrange(len(man))))
            except ValueError:
                g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(man.pop(randrange(len(man))))
        except ValueError:
            b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))
    else:
        g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))

In [ ]:
a = 1
t = 0
for i in [boy,girl,man,woman,wife,husband,widowed,others]:
    r=0
    if len(i)!=0:
        for j in i:
            if len(h1)!=0:
                h = h1.pop(randrange(len((h1))))
                t+=1
                r+=1
                j = list(j)
                j.append(h)
                j.append(0)
                final.append(j)
                house_res_id.append((h,j[0]))
            else:
                t+=1
                r+=1
                j = list(j)
                j.append(0)
                j.append(0)
                final.append(j)
        print(a,r,t)
        a+=1
    else:
        pass

In [ ]:
# %time
                                                    ########## Village9 ################
boy = sample(v9_male_child,len(v9_male_child))
girl = sample(v9_female_child,len(v9_female_child))
husband = sample(v9_male_adult_married,len(v9_male_adult_married))
wife = sample(v9_female_adult_married,len(v9_female_adult_married))
man = sample(v9_male_adult_unmarried,len(v9_male_adult_unmarried))
woman = sample(v9_female_adult_unmarried,len(v9_female_adult_unmarried))
widowed = sample(v9_widowed,len(v9_widowed))
others = sample(v9_others,len(v9_others))
house = sample(v9,len(v9))

In [ ]:
h1 = []
count = 1
while len(house)!=0:
    print(count)
    count+=1
    try:
        for i in range(len(house)):                                      
            h = house.pop(randrange(len(house)))
            ch = choice(range(10))
            if ch in [1,2]:                                              #3
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                            else:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                    else:
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g = list(woman.pop(randrange(len(woman))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(man.pop(randrange(len(man))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                else:
                    w = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w
                    w.append(h)
                    w.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
            elif ch in [3,4,5]:                                                     #5
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    o = list(others.pop(randrange(len(others))))
                    o.append(h)
                    o.append(0)
                    o.append(0)
                    o.append(0)
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,g,b,g1))
                            else:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,g,b,b1))
                        elif choice(range(1))==1:
                            if len(man)!=0:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w1,w,g,b,b1))
                            else:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w1,w,g,b,g1))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))
                            else:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                            else:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))                        
            else:                                                 #4
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,b,g))
    except ValueError:
        print(h)
        h1.append(h)
        pass
while len(h1)!=0:
    h = h1.pop(randrange(len(h1)))
    try:
        w = list(widowed.pop(randrange(len(widowed))))
    except ValueError:
        h1.append(h)
        print('last done')
        print(h)
        break
    x,y,z,a,b = w
    w.append(h)
    w.append(0)
    w.append(0)
    w.append(0)
    house_res_id.append((h,x))
    if a>45:
        try:
            g = list(woman.pop(randrange(len(woman))))
        except ValueError:
            try:
                g = list(man.pop(randrange(len(man))))
            except ValueError:
                g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(man.pop(randrange(len(man))))
        except ValueError:
            b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))
    else:
        g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))

In [ ]:
a = 1
t = 0
for i in [boy,girl,man,woman,wife,husband,widowed,others]:
    r=0
    if len(i)!=0:
        for j in i:
            if len(h1)!=0:
                h = h1.pop(randrange(len((h1))))
                t+=1
                r+=1
                j = list(j)
                j.append(h)
                j.append(0)
                final.append(j)
                house_res_id.append((h,j[0]))
            else:
                t+=1
                r+=1
                j = list(j)
                j.append(0)
                j.append(0)
                final.append(j)
        print(a,r,t)
        a+=1
    else:
        pass

In [ ]:
# %time
                                                    ########## Village10 ################
boy = sample(v10_male_child,len(v10_male_child))
girl = sample(v10_female_child,len(v10_female_child))
husband = sample(v10_male_adult_married,len(v10_male_adult_married))
wife = sample(v10_female_adult_married,len(v10_female_adult_married))
man = sample(v10_male_adult_unmarried,len(v10_male_adult_unmarried))
woman = sample(v10_female_adult_unmarried,len(v10_female_adult_unmarried))
widowed = sample(v10_widowed,len(v10_widowed))
others = sample(v10_others,len(v10_others))
house = sample(v10,len(v10))

In [ ]:
h1 = []
count = 1
while len(house)!=0:
    print(count)
    count+=1
    try:
        for i in range(len(house)):                                      
            h = house.pop(randrange(len(house)))
            ch = choice(range(10))
            if ch in [1,2]:                                              #3
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                            else:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                    else:
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g = list(woman.pop(randrange(len(woman))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(man.pop(randrange(len(man))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                else:
                    w = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w
                    w.append(h)
                    w.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
            elif ch in [3,4,5]:                                                     #5
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    o = list(others.pop(randrange(len(others))))
                    o.append(h)
                    o.append(0)
                    o.append(0)
                    o.append(0)
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,g,b,g1))
                            else:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,g,b,b1))
                        elif choice(range(1))==1:
                            if len(man)!=0:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w1,w,g,b,b1))
                            else:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w1,w,g,b,g1))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))
                            else:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                            else:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))                        
            else:                                                 #4
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,b,g))
    except ValueError:
        print(h)
        h1.append(h)
        pass
while len(h1)!=0:
    h = h1.pop(randrange(len(h1)))
    try:
        w = list(widowed.pop(randrange(len(widowed))))
    except ValueError:
        h1.append(h)
        print('last done')
        print(h)
        break
    x,y,z,a,b = w
    w.append(h)
    w.append(0)
    w.append(0)
    w.append(0)
    house_res_id.append((h,x))
    if a>45:
        try:
            g = list(woman.pop(randrange(len(woman))))
        except ValueError:
            try:
                g = list(man.pop(randrange(len(man))))
            except ValueError:
                g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(man.pop(randrange(len(man))))
        except ValueError:
            b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))
    else:
        try:
            g = list(girl.pop(randrange(len(girl))))
        except ValueError:
            try:
                g = list(boy.pop(randrange(len(boy))))
            except ValueError:
                g = list(woman.pop(randrange(len(woman))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(boy.pop(randrange(len(boy))))
        except ValueError:
            try:
                b = list(girl.pop(randrange(len(girl))))
            except ValueError:
                try:
                    b = list(man.pop(randrange(len(girl))))
                except ValueError:
                    b = list(woman.pop(randrange(len(woman))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))

In [ ]:
a = 1
t = 0
for i in [boy,girl,man,woman,wife,husband,widowed,others]:
    r=0
    if len(i)!=0:
        for j in i:
            if len(h1)!=0:
                h = h1.pop(randrange(len((h1))))
                t+=1
                r+=1
                j = list(j)
                j.append(h)
                j.append(0)
                final.append(j)
                house_res_id.append((h,j[0]))
            else:
                t+=1
                r+=1
                j = list(j)
                j.append(0)
                j.append(0)
                final.append(j)
        print(a,r,t)
        a+=1
    else:
        pass

In [ ]:
# %time
                                                    ########## Village11 ################
boy = sample(v11_male_child,len(v11_male_child))
girl = sample(v11_female_child,len(v11_female_child))
husband = sample(v11_male_adult_married,len(v11_male_adult_married))
wife = sample(v11_female_adult_married,len(v11_female_adult_married))
man = sample(v11_male_adult_unmarried,len(v11_male_adult_unmarried))
woman = sample(v11_female_adult_unmarried,len(v11_female_adult_unmarried))
widowed = sample(v11_widowed,len(v11_widowed))
others = sample(v11_others,len(v11_others))
house = sample(v11,len(v11))

In [ ]:
h1 = []
count = 1
while len(house)!=0:
    print(count)
    count+=1
    try:
        for i in range(len(house)):                                      
            h = house.pop(randrange(len(house)))
            ch = choice(range(10))
            if ch in [1,2]:                                              #3
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                            else:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                    else:
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g = list(woman.pop(randrange(len(woman))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(man.pop(randrange(len(man))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                else:
                    w = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w
                    w.append(h)
                    w.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
            elif ch in [3,4,5]:                                                     #5
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    o = list(others.pop(randrange(len(others))))
                    o.append(h)
                    o.append(0)
                    o.append(0)
                    o.append(0)
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,g,b,g1))
                            else:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,g,b,b1))
                        elif choice(range(1))==1:
                            if len(man)!=0:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w1,w,g,b,b1))
                            else:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w1,w,g,b,g1))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))
                            else:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                            else:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))                        
            else:                                                 #4
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,b,g))
    except ValueError:
        print(h)
        h1.append(h)
        pass
while len(h1)!=0:
    h = h1.pop(randrange(len(h1)))
    try:
        w = list(widowed.pop(randrange(len(widowed))))
    except ValueError:
        h1.append(h)
        print('last done')
        print(h)
        break
    x,y,z,a,b = w
    w.append(h)
    w.append(0)
    w.append(0)
    w.append(0)
    house_res_id.append((h,x))
    if a>45:
        try:
            g = list(woman.pop(randrange(len(woman))))
        except ValueError:
            try:
                g = list(man.pop(randrange(len(man))))
            except ValueError:
                g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(man.pop(randrange(len(man))))
        except ValueError:
            b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))
    else:
        g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))

In [ ]:
a = 1
t = 0
for i in [boy,girl,man,woman,wife,husband,widowed,others]:
    r=0
    if len(i)!=0:
        for j in i:
            if len(h1)!=0:
                h = h1.pop(randrange(len((h1))))
                t+=1
                r+=1
                j = list(j)
                j.append(h)
                j.append(0)
                final.append(j)
                house_res_id.append((h,j[0]))
            else:
                t+=1
                r+=1
                j = list(j)
                j.append(0)
                j.append(0)
                final.append(j)
        print(a,r,t)
        a+=1
    else:
        pass

In [ ]:
# %time
                                                    ########## Village12 ################
boy = sample(v12_male_child,len(v12_male_child))
girl = sample(v12_female_child,len(v12_female_child))
husband = sample(v12_male_adult_married,len(v12_male_adult_married))
wife = sample(v12_female_adult_married,len(v12_female_adult_married))
man = sample(v12_male_adult_unmarried,len(v12_male_adult_unmarried))
woman = sample(v12_female_adult_unmarried,len(v12_female_adult_unmarried))
widowed = sample(v12_widowed,len(v12_widowed))
others = sample(v12_others,len(v12_others))
house = sample(v12,len(v12))

In [ ]:
h1 = []
count = 1
while len(house)!=0:
    print(count)
    count+=1
    try:
        for i in range(len(house)):                                      
            h = house.pop(randrange(len(house)))
            ch = choice(range(10))
            if ch in [1,2]:                                              #3
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                            else:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                    else:
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g = list(woman.pop(randrange(len(woman))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(man.pop(randrange(len(man))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                else:
                    w = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w
                    w.append(h)
                    w.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
            elif ch in [3,4,5]:                                                     #5
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    o = list(others.pop(randrange(len(others))))
                    o.append(h)
                    o.append(0)
                    o.append(0)
                    o.append(0)
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,g,b,g1))
                            else:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,g,b,b1))
                        elif choice(range(1))==1:
                            if len(man)!=0:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w1,w,g,b,b1))
                            else:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w1,w,g,b,g1))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))
                            else:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                            else:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))                        
            else:                                                 #4
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,b,g))
    except ValueError:
        print(h)
        h1.append(h)
        pass
while len(h1)!=0:
    h = h1.pop(randrange(len(h1)))
    try:
        w = list(widowed.pop(randrange(len(widowed))))
    except ValueError:
        h1.append(h)
        print('last done')
        print(h)
        break
    x,y,z,a,b = w
    w.append(h)
    w.append(0)
    w.append(0)
    w.append(0)
    house_res_id.append((h,x))
    if a>45:
        try:
            g = list(woman.pop(randrange(len(woman))))
        except ValueError:
            try:
                g = list(man.pop(randrange(len(man))))
            except ValueError:
                g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(man.pop(randrange(len(man))))
        except ValueError:
            b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))
    else:
        try:
            g = list(girl.pop(randrange(len(girl))))
        except ValueError:
            try:
                g = list(boy.pop(randrange(len(boy))))
            except ValueError:
                g = list(woman.pop(randrange(len(woman))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(boy.pop(randrange(len(boy))))
        except ValueError:
            try:
                b = list(girl.pop(randrange(len(girl))))
            except ValueError:
                try:
                    b = list(man.pop(randrange(len(girl))))
                except ValueError:
                    b = list(woman.pop(randrange(len(woman))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))

In [ ]:
a = 1
t = 0
for i in [boy,girl,man,woman,wife,husband,widowed,others]:
    r=0
    if len(i)!=0:
        for j in i:
            if len(h1)!=0:
                h = h1.pop(randrange(len((h1))))
                t+=1
                r+=1
                j = list(j)
                j.append(h)
                j.append(0)
                final.append(j)
                house_res_id.append((h,j[0]))
            else:
                t+=1
                r+=1
                j = list(j)
                j.append(0)
                j.append(0)
                final.append(j)
        print(a,r,t)
        a+=1
    else:
        pass

In [ ]:
# %time
                                                    ########## Village13 ################
boy = sample(v13_male_child,len(v13_male_child))
girl = sample(v13_female_child,len(v13_female_child))
husband = sample(v13_male_adult_married,len(v13_male_adult_married))
wife = sample(v13_female_adult_married,len(v13_female_adult_married))
man = sample(v13_male_adult_unmarried,len(v13_male_adult_unmarried))
woman = sample(v13_female_adult_unmarried,len(v13_female_adult_unmarried))
widowed = sample(v13_widowed,len(v13_widowed))
others = sample(v13_others,len(v13_others))
house = sample(v13,len(v13))

In [ ]:
h1 = []
count = 1
while len(house)!=0:
    print(count)
    count+=1
    try:
        for i in range(len(house)):                                      
            h = house.pop(randrange(len(house)))
            ch = choice(range(10))
            if ch in [1,2]:                                              #3
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                            else:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                    else:
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g = list(woman.pop(randrange(len(woman))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(man.pop(randrange(len(man))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                else:
                    w = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w
                    w.append(h)
                    w.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
            elif ch in [3,4,5]:                                                     #5
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    o = list(others.pop(randrange(len(others))))
                    o.append(h)
                    o.append(0)
                    o.append(0)
                    o.append(0)
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,g,b,g1))
                            else:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,g,b,b1))
                        elif choice(range(1))==1:
                            if len(man)!=0:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w1,w,g,b,b1))
                            else:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w1,w,g,b,g1))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))
                            else:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                            else:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))                        
            else:                                                 #4
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,b,g))
    except ValueError:
        print(h)
        h1.append(h)
        pass
while len(h1)!=0:
    h = h1.pop(randrange(len(h1)))
    try:
        w = list(widowed.pop(randrange(len(widowed))))
    except ValueError:
        h1.append(h)
        print('last done')
        print(h)
        break
    x,y,z,a,b = w
    w.append(h)
    w.append(0)
    w.append(0)
    w.append(0)
    house_res_id.append((h,x))
    if a>45:
        try:
            g = list(woman.pop(randrange(len(woman))))
        except ValueError:
            try:
                g = list(man.pop(randrange(len(man))))
            except ValueError:
                g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(man.pop(randrange(len(man))))
        except ValueError:
            b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))
    else:
        try:
            g = list(girl.pop(randrange(len(girl))))
        except ValueError:
            g = list(boy.pop(randrange(len(boy))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(boy.pop(randrange(len(boy))))
        except ValueError:
            try:
                b = list(girl.pop(randrange(len(girl))))
            except ValuelError:
                b = list(man.pop(randrange(len(girl))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))

In [ ]:
a = 1
t = 0
for i in [boy,girl,man,woman,wife,husband,widowed,others]:
    r=0
    if len(i)!=0:
        for j in i:
            if len(h1)!=0:
                h = h1.pop(randrange(len((h1))))
                t+=1
                r+=1
                j = list(j)
                j.append(h)
                j.append(0)
                final.append(j)
                house_res_id.append((h,j[0]))
            else:
                t+=1
                r+=1
                j = list(j)
                j.append(0)
                j.append(0)
                final.append(j)
        print(a,r,t)
        a+=1
    else:
        pass

In [ ]:
# %time
                                                    ########## Village14 ################
boy = sample(v14_male_child,len(v14_male_child))
girl = sample(v14_female_child,len(v14_female_child))
husband = sample(v14_male_adult_married,len(v14_male_adult_married))
wife = sample(v14_female_adult_married,len(v14_female_adult_married))
man = sample(v14_male_adult_unmarried,len(v14_male_adult_unmarried))
woman = sample(v14_female_adult_unmarried,len(v14_female_adult_unmarried))
widowed = sample(v14_widowed,len(v14_widowed))
others = sample(v14_others,len(v14_others))
house = sample(v14,len(v14))

In [ ]:
h1 = []
count = 1
while len(house)!=0:
    print(count)
    count+=1
    try:
        for i in range(len(house)):                                      
            h = house.pop(randrange(len(house)))
            ch = choice(range(10))
            if ch in [1,2]:                                              #3
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                            else:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                    else:
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g = list(woman.pop(randrange(len(woman))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(man.pop(randrange(len(man))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                else:
                    w = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w
                    w.append(h)
                    w.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
            elif ch in [3,4,5]:                                                     #5
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    o = list(others.pop(randrange(len(others))))
                    o.append(h)
                    o.append(0)
                    o.append(0)
                    o.append(0)
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,g,b,g1))
                            else:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,g,b,b1))
                        elif choice(range(1))==1:
                            if len(man)!=0:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w1,w,g,b,b1))
                            else:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w1,w,g,b,g1))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))
                            else:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                            else:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))                        
            else:                                                 #4
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,b,g))
    except ValueError:
        print(h)
        h1.append(h)
        pass
while len(h1)!=0:
    h = h1.pop(randrange(len(h1)))
    try:
        w = list(widowed.pop(randrange(len(widowed))))
    except ValueError:
        h1.append(h)
        print('last done')
        print(h)
        break
    x,y,z,a,b = w
    w.append(h)
    w.append(0)
    w.append(0)
    w.append(0)
    house_res_id.append((h,x))
    if a>45:
        try:
            g = list(woman.pop(randrange(len(woman))))
        except ValueError:
            try:
                g = list(man.pop(randrange(len(man))))
            except ValueError:
                g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(man.pop(randrange(len(man))))
        except ValueError:
            b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))
    else:
        try:
            g = list(girl.pop(randrange(len(girl))))
        except ValueError:
            g = list(boy.pop(randrange(len(boy))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(boy.pop(randrange(len(boy))))
        except ValueError:
            try:
                b = list(girl.pop(randrange(len(girl))))
            except ValuelError:
                b = list(man.pop(randrange(len(girl))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))

In [ ]:
a = 1
t = 0
for i in [boy,girl,man,woman,wife,husband,widowed,others]:
    r=0
    if len(i)!=0:
        for j in i:
            if len(h1)!=0:
                h = h1.pop(randrange(len((h1))))
                t+=1
                r+=1
                j = list(j)
                j.append(h)
                j.append(0)
                final.append(j)
                house_res_id.append((h,j[0]))
            else:
                t+=1
                r+=1
                j = list(j)
                j.append(0)
                j.append(0)
                final.append(j)
        print(a,r,t)
        a+=1
    else:
        pass

In [ ]:
%time
                                                    ########## Village15 ################
boy = sample(v15_male_child,len(v15_male_child))
girl = sample(v15_female_child,len(v15_female_child))
husband = sample(v15_male_adult_married,len(v15_male_adult_married))
wife = sample(v15_female_adult_married,len(v15_female_adult_married))
man = sample(v15_male_adult_unmarried,len(v15_male_adult_unmarried))
woman = sample(v15_female_adult_unmarried,len(v15_female_adult_unmarried))
widowed = sample(v15_widowed,len(v15_widowed))
others = sample(v15_others,len(v15_others))
house = sample(v15,len(v15))

In [ ]:
h1 = []
count = 1
while len(house)!=0:
    print(count)
    count+=1
    try:
        for i in range(len(house)):                                      
            h = house.pop(randrange(len(house)))
            ch = choice(range(10))
            if ch in [1,2]:                                              #3
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                            else:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                    else:
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g = list(woman.pop(randrange(len(woman))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(man.pop(randrange(len(man))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                else:
                    w = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w
                    w.append(h)
                    w.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
            elif ch in [3,4,5]:                                                     #5
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    o = list(others.pop(randrange(len(others))))
                    o.append(h)
                    o.append(0)
                    o.append(0)
                    o.append(0)
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,g,b,g1))
                            else:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,g,b,b1))
                        elif choice(range(1))==1:
                            if len(man)!=0:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w1,w,g,b,b1))
                            else:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w1,w,g,b,g1))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))
                            else:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                            else:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))                        
            else:                                                 #4
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,b,g))
    except ValueError:
        print(h)
        h1.append(h)
        pass
while len(h1)!=0:
    h = h1.pop(randrange(len(h1)))
    try:
        w = list(widowed.pop(randrange(len(widowed))))
    except ValueError:
        h1.append(h)
        print('last done')
        print(h)
        break
    x,y,z,a,b = w
    w.append(h)
    w.append(0)
    w.append(0)
    w.append(0)
    house_res_id.append((h,x))
    if a>45:
        try:
            g = list(woman.pop(randrange(len(woman))))
        except ValueError:
            try:
                g = list(man.pop(randrange(len(man))))
            except ValueError:
                g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(man.pop(randrange(len(man))))
        except ValueError:
            b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))
    else:
        try:
            g = list(girl.pop(randrange(len(girl))))
        except ValueError:
            g = list(boy.pop(randrange(len(boy))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(boy.pop(randrange(len(boy))))
        except ValueError:
            try:
                b = list(girl.pop(randrange(len(girl))))
            except ValuelError:
                b = list(man.pop(randrange(len(girl))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))

In [ ]:
a = 1
t = 0
for i in [boy,girl,man,woman,wife,husband,widowed,others]:
    r=0
    if len(i)!=0:
        for j in i:
            if len(h1)!=0:
                h = h1.pop(randrange(len((h1))))
                t+=1
                r+=1
                j = list(j)
                j.append(h)
                j.append(0)
                final.append(j)
                house_res_id.append((h,j[0]))
            else:
                t+=1
                r+=1
                j = list(j)
                j.append(0)
                j.append(0)
                final.append(j)
        print(a,r,t)
        a+=1
    else:
        pass

In [ ]:
%time
                                                    ########## Village16 ################
boy = sample(v16_male_child,len(v16_male_child))
girl = sample(v16_female_child,len(v16_female_child))
husband = sample(v16_male_adult_married,len(v16_male_adult_married))
wife = sample(v16_female_adult_married,len(v16_female_adult_married))
man = sample(v16_male_adult_unmarried,len(v16_male_adult_unmarried))
woman = sample(v16_female_adult_unmarried,len(v16_female_adult_unmarried))
widowed = sample(v16_widowed,len(v16_widowed))
others = sample(v16_others,len(v16_others))
house = sample(v16,len(v16))

In [ ]:
h1 = []
count = 1
while len(house)!=0:
    print(count)
    count+=1
    try:
        for i in range(len(house)):                                      
            h = house.pop(randrange(len(house)))
            ch = choice(range(10))
            if ch in [1,2]:                                              #3
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                            else:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                    else:
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g = list(woman.pop(randrange(len(woman))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(man.pop(randrange(len(man))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                else:
                    w = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w
                    w.append(h)
                    w.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
            elif ch in [3,4,5]:                                                     #5
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    o = list(others.pop(randrange(len(others))))
                    o.append(h)
                    o.append(0)
                    o.append(0)
                    o.append(0)
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,g,b,g1))
                            else:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,g,b,b1))
                        elif choice(range(1))==1:
                            if len(man)!=0:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w1,w,g,b,b1))
                            else:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w1,w,g,b,g1))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))
                            else:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                            else:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))                        
            else:                                                 #4
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,b,g))
    except ValueError:
        print(h)
        h1.append(h)
        pass
while len(h1)!=0:
    h = h1.pop(randrange(len(h1)))
    try:
        w = list(widowed.pop(randrange(len(widowed))))
    except ValueError:
        h1.append(h)
        print('last done')
        print(h)
        break
    x,y,z,a,b = w
    w.append(h)
    w.append(0)
    w.append(0)
    w.append(0)
    house_res_id.append((h,x))
    if a>45:
        try:
            g = list(woman.pop(randrange(len(woman))))
        except ValueError:
            try:
                g = list(man.pop(randrange(len(man))))
            except ValueError:
                g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(man.pop(randrange(len(man))))
        except ValueError:
            b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))
    else:
        try:
            g = list(girl.pop(randrange(len(girl))))
        except ValueError:
            g = list(boy.pop(randrange(len(boy))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(boy.pop(randrange(len(boy))))
        except ValueError:
            try:
                b = list(girl.pop(randrange(len(girl))))
            except ValuelError:
                b = list(man.pop(randrange(len(girl))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))

In [ ]:
a = 1
t = 0
for i in [boy,girl,man,woman,wife,husband,widowed,others]:
    r=0
    if len(i)!=0:
        for j in i:
            if len(h1)!=0:
                h = h1.pop(randrange(len((h1))))
                t+=1
                r+=1
                j = list(j)
                j.append(h)
                j.append(0)
                final.append(j)
                house_res_id.append((h,j[0]))
            else:
                t+=1
                r+=1
                j = list(j)
                j.append(0)
                j.append(0)
                final.append(j)
        print(a,r,t)
        a+=1
    else:
        pass

In [ ]:
%time
                                                    ########## Village17 ################
boy = sample(v17_male_child,len(v17_male_child))
girl = sample(v17_female_child,len(v17_female_child))
husband = sample(v17_male_adult_married,len(v17_male_adult_married))
wife = sample(v17_female_adult_married,len(v17_female_adult_married))
man = sample(v17_male_adult_unmarried,len(v17_male_adult_unmarried))
woman = sample(v17_female_adult_unmarried,len(v17_female_adult_unmarried))
widowed = sample(v17_widowed,len(v17_widowed))
others = sample(v17_others,len(v17_others))
house = sample(v17,len(v17))

In [ ]:
h1 = []
count = 1
while len(house)!=0:
    print(count)
    count+=1
    try:
        for i in range(len(house)):                                      
            h = house.pop(randrange(len(house)))
            ch = choice(range(10))
            if ch in [1,2]:                                              #3
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b = list(boy.pop(randrange(len(boy))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                            else:
                                g = list(girl.pop(randrange(len(girl))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                    else:
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g = list(woman.pop(randrange(len(woman))))
                                g.append(h)
                                g.append(0)
                                g.append(x)
                                g.append(x1)
                                final.extend((hu,wi,g))
                            else:
                                b = list(man.pop(randrange(len(man))))
                                b.append(h)
                                b.append(0)
                                b.append(x)
                                b.append(x1)
                                final.extend((hu,wi,b))
                else:
                    w = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w
                    w.append(h)
                    w.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,g,b))
            elif ch in [3,4,5]:                                                     #5
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    o = list(others.pop(randrange(len(others))))
                    o.append(h)
                    o.append(0)
                    o.append(0)
                    o.append(0)
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g,o))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(woman)!=0:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,g,b,g1))
                            else:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,g,b,b1))
                        elif choice(range(1))==1:
                            if len(man)!=0:
                                b1 = list(man.pop(randrange(len(man))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w1,w,g,b,b1))
                            else:
                                g1 = list(woman.pop(randrange(len(woman))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w1,w,g,b,g1))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        if choice(range(1))==0:
                            if len(girl)!=0:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))
                            else:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                        elif choice(range(1))==1:
                            if len(boy)!=0:
                                b1 = list(boy.pop(randrange(len(boy))))
                                b1.append(h)
                                b1.append(0)
                                b1.append(x)
                                b1.append(x1)
                                final.extend((w,w1,b,g,b1))
                            else:
                                g1 = list(girl.pop(randrange(len(girl))))
                                g1.append(h)
                                g1.append(0)
                                g1.append(x)
                                g1.append(x1)
                                final.extend((w,w1,b,g,g1))                        
            else:                                                 #4
                if len(husband)!=0 and len(wife)!=0:
                    hu = list(husband.pop(randrange(len(husband))))
                    wi = list(wife.pop(randrange(len(wife))))
                    x,y,z,a,b = hu
                    x1,y1,z1,a1,b1 = wi
                    hu.append(h)
                    hu.append(x1)
                    wi.append(h)
                    wi.append(x)
                    wi.append(0)
                    wi.append(0)
                    hu.append(0)
                    hu.append(0)
                    house_res_id.append((h,x))
                    if a<45:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                    else:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((hu,wi,b,g))
                else:
                    w = list(others.pop(randrange(len(others))))
                    w1 = list(widowed.pop(randrange(len(widowed))))
                    x,y,z,a,b = w1
                    w.append(h)
                    w.append(0)
                    w1.append(h)
                    w1.append(0)
                    w1.append(0)
                    w1.append(0)
                    w.append(0)
                    w.append(0)
                    house_res_id.append((h,x))
                    if a>45:
                        g = list(woman.pop(randrange(len(woman))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(man.pop(randrange(len(man))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,g,b))
                    else:
                        g = list(girl.pop(randrange(len(girl))))
                        g.append(h)
                        g.append(0)
                        g.append(x)
                        g.append(x1)
                        b = list(boy.pop(randrange(len(boy))))
                        b.append(h)
                        b.append(0)
                        b.append(x)
                        b.append(x1)
                        final.extend((w,w1,b,g))
    except ValueError:
        print(h)
        h1.append(h)
        pass
while len(h1)!=0:
    h = h1.pop(randrange(len(h1)))
    try:
        w = list(widowed.pop(randrange(len(widowed))))
    except ValueError:
        h1.append(h)
        print('last done')
        print(h)
        break
    x,y,z,a,b = w
    w.append(h)
    w.append(0)
    w.append(0)
    w.append(0)
    house_res_id.append((h,x))
    if a>45:
        try:
            g = list(woman.pop(randrange(len(woman))))
        except ValueError:
            try:
                g = list(man.pop(randrange(len(man))))
            except ValueError:
                g = list(girl.pop(randrange(len(girl))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(man.pop(randrange(len(man))))
        except ValueError:
            b = list(boy.pop(randrange(len(boy))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))
    else:
        try:
            g = list(girl.pop(randrange(len(girl))))
        except ValueError:
            try:
                g = list(boy.pop(randrange(len(boy))))
            except ValueError:
                g = list(woman.pop(randrange(len(woman))))
        g.append(h)
        g.append(0)
        g.append(x)
        g.append(x1)
        try:
            b = list(boy.pop(randrange(len(boy))))
        except ValueError:
            try:
                b = list(girl.pop(randrange(len(girl))))
            except ValueError:
                try:
                    b = list(man.pop(randrange(len(girl))))
                except ValueError:
                    b = list(woman.pop(randrange(len(woman))))
        b.append(h)
        b.append(0)
        b.append(x)
        b.append(x1)
        final.extend((w,g,b))

In [ ]:
a = 1
t = 0
for i in [boy,girl,man,woman,wife,husband,widowed,others]:
    r=0
    if len(i)!=0:
        for j in i:
            if len(h1)!=0:
                h = h1.pop(randrange(len((h1))))
                t+=1
                r+=1
                j = list(j)
                j.append(h)
                j.append(0)
                final.append(j)
                house_res_id.append((h,j[0]))
            else:
                t+=1
                r+=1
                j = list(j)
                j.append(0)
                j.append(0)
                final.append(j)
        print(a,r,t)
        a+=1
    else:
        pass

In [ ]:
len(final),len(house_res_id)

In [ ]:
%time 
q=[]
p=[]
for x in final:
    q.append(x[0])
for i in range(1,len(r_id)+1):
    if i not in q:
        p.append(i)

In [ ]:
len(q),len(r_id)

# House_ID for Residents

In [ ]:
#r_id,rv_id,marital,age,rsex,rh_id,sp_id,fath_id,mom_id

rv_id = []
r_id = []
marital =[]
age = []
rsex =[]
rh_id = []
fath_id = []
mom_id = []

#r_id,sp_id
# res_spouse_id = []
r_id = []
sp_id = []

#res_id,house_id
# house_res_id =[]
res_id = []
# h_id

In [ ]:
li=final
final = []
for i in li:
    if len(i)==6:
        i.append(0)
        i.append(0)
        i.append(0)
        final.append(i)
    if len(i)==7:
        i.append(0)
        i.append(0)
        final.append(i)
#     elif len(i)==8:
#         i.pop(-1)
#         final.append(i)
    else:
        final.append(i)

In [ ]:
final.sort()
for x,y,z,a,b,c,d,e,f in final:
    r_id.append(x)
    rv_id.append(y)
    marital.append(z)
    age.append(a)
    rsex.append(b)
    rh_id.append(c)
    sp_id.append(d)
    fath_id.append(e)
    mom_id.append(f)
len(r_id)

In [ ]:
sp_name = []
li = list(zip(r_id,rfirst_name))
for i in sp_id:
    if i !=0:
        for x,y in li:
            if i == x:
                sp_name.append(y)
                break
    else:
        sp_name.append('')
len(sp_name)

In [ ]:
h_id = []
res_id = []

In [ ]:
house_res_id.sort()
for x,y in house_res_id:
    h_id.append(x)
    res_id.append(y)

In [ ]:
x,y,z,house = random_village_region_street_house(tcount)
house = sample(house,len(house))
len(house),len(x),len(y),len(z)

In [ ]:
for i in range(len(house)-len(h_id)):
    house.pop()
len(house)

In [ ]:
pd.read_csv('./data/gpdata/village.csv')['VillageName']

In [ ]:
pr_no = []
i = 0
while i < len(h_id):
    if hgp_id[i]==1:
        pr_no.append('ak'+'-'+house[i]+str(res_id[i]))
    elif hgp_id[i]==2:
        pr_no.append('np'+'-'+house[i]+str(res_id[i]))
    elif hgp_id[i]==3:
        pr_no.append('de'+'-'+house[i]+str(res_id[i]))
    elif hgp_id[i]==4:
        pr_no.append('rtr'+'-'+house[i]+str(res_id[i]))
    elif hgp_id[i]==5:
        pr_no.append('agd'+'-'+house[i]+str(res_id[i]))
    elif hgp_id[i]==6:
        pr_no.append('cca'+'-'+house[i]+str(res_id[i]))
    elif hgp_id[i]==7:
        pr_no.append('kk'+'-'+house[i]+str(res_id[i]))
    elif hgp_id[i]==8:
        pr_no.append('vr'+'-'+house[i]+str(res_id[i]))
    elif hgp_id[i]==9:
        pr_no.append('adk'+'-'+house[i]+str(res_id[i]))
    elif hgp_id[i]==10:
        pr_no.append('pp'+'-'+house[i]+str(res_id[i]))
    elif hgp_id[i]==11:
        pr_no.append('bg'+'-'+house[i]+str(res_id[i]))
    elif hgp_id[i]==12:
        pr_no.append('ar'+'-'+house[i]+str(res_id[i]))
    elif hgp_id[i]==13:
        pr_no.append('jg'+'-'+house[i]+str(res_id[i]))
    elif hgp_id[i]==14:
        pr_no.append('ib'+'-'+house[i]+str(res_id[i]))
    elif hgp_id[i]==15:
        pr_no.append('ap'+'-'+house[i]+str(res_id[i]))
    elif hgp_id[i]==16:
        pr_no.append('tbd'+'-'+house[i]+str(res_id[i]))
    elif hgp_id[i]==17:
        pr_no.append('bh'+'-'+house[i]+str(res_id[i]))    
    i+=1
len(pr_no)

In [ ]:
reg_no = []
a = range(0,10)
var = 'A'
alpha=[(chr(ord(var)+i)) for i in range(26)]
alpha = sample(alpha,26)
i = 0
while i < len(h_id):
    if hgp_id[i]==1:
        reg_no.append('AK'+'-'+choice(alpha)+str(choice(a))+choice(alpha)+str(choice(a))+str(choice(a))+str(choice(a))+str(choice(a))+choice(alpha)+choice(alpha)+str(choice(a))+str(choice(a))+choice(alpha))
    elif hgp_id[i]==2:
        reg_no.append('NP'+'-'+choice(alpha)+str(choice(a))+choice(alpha)+str(choice(a))+str(choice(a))+str(choice(a))+str(choice(a))+choice(alpha)+choice(alpha)+str(choice(a))+str(choice(a))+choice(alpha))
    elif hgp_id[i]==3:
        reg_no.append('DE'+'-'+choice(alpha)+str(choice(a))+choice(alpha)+str(choice(a))+str(choice(a))+str(choice(a))+str(choice(a))+choice(alpha)+choice(alpha)+str(choice(a))+str(choice(a))+choice(alpha))
    elif hgp_id[i]==4:
        reg_no.append('RTR'+'-'+choice(alpha)+str(choice(a))+choice(alpha)+str(choice(a))+str(choice(a))+str(choice(a))+str(choice(a))+choice(alpha)+choice(alpha)+str(choice(a))+str(choice(a))+choice(alpha))
    elif hgp_id[i]==5:
        reg_no.append('AGD'+'-'+choice(alpha)+str(choice(a))+choice(alpha)+str(choice(a))+str(choice(a))+str(choice(a))+str(choice(a))+choice(alpha)+choice(alpha)+str(choice(a))+str(choice(a))+choice(alpha))
    elif hgp_id[i]==6:
        reg_no.append('CCA'+'-'+choice(alpha)+str(choice(a))+choice(alpha)+str(choice(a))+str(choice(a))+str(choice(a))+str(choice(a))+choice(alpha)+choice(alpha)+str(choice(a))+str(choice(a))+choice(alpha))
    elif hgp_id[i]==7:
        reg_no.append('KK'+'-'+choice(alpha)+str(choice(a))+choice(alpha)+str(choice(a))+str(choice(a))+str(choice(a))+str(choice(a))+choice(alpha)+choice(alpha)+str(choice(a))+str(choice(a))+choice(alpha))
    elif hgp_id[i]==8:
        reg_no.append('VR'+'-'+choice(alpha)+str(choice(a))+choice(alpha)+str(choice(a))+str(choice(a))+str(choice(a))+str(choice(a))+choice(alpha)+choice(alpha)+str(choice(a))+str(choice(a))+choice(alpha))
    elif hgp_id[i]==9:
        reg_no.append('ADK'+'-'+choice(alpha)+str(choice(a))+choice(alpha)+str(choice(a))+str(choice(a))+str(choice(a))+str(choice(a))+choice(alpha)+choice(alpha)+str(choice(a))+str(choice(a))+choice(alpha))
    elif hgp_id[i]==10:
        reg_no.append('PP'+'-'+choice(alpha)+str(choice(a))+choice(alpha)+str(choice(a))+str(choice(a))+str(choice(a))+str(choice(a))+choice(alpha)+choice(alpha)+str(choice(a))+str(choice(a))+choice(alpha))
    elif hgp_id[i]==11:
        reg_no.append('BG'+'-'+choice(alpha)+str(choice(a))+choice(alpha)+str(choice(a))+str(choice(a))+str(choice(a))+str(choice(a))+choice(alpha)+choice(alpha)+str(choice(a))+str(choice(a))+choice(alpha))
    elif hgp_id[i]==12:
        reg_no.append('AR'+'-'+choice(alpha)+str(choice(a))+choice(alpha)+str(choice(a))+str(choice(a))+str(choice(a))+str(choice(a))+choice(alpha)+choice(alpha)+str(choice(a))+str(choice(a))+choice(alpha))
    elif hgp_id[i]==13:
        reg_no.append('JG'+'-'+choice(alpha)+str(choice(a))+choice(alpha)+str(choice(a))+str(choice(a))+str(choice(a))+str(choice(a))+choice(alpha)+choice(alpha)+str(choice(a))+str(choice(a))+choice(alpha))
    elif hgp_id[i]==14:
        reg_no.append('IB'+'-'+choice(alpha)+str(choice(a))+choice(alpha)+str(choice(a))+str(choice(a))+str(choice(a))+str(choice(a))+choice(alpha)+choice(alpha)+str(choice(a))+str(choice(a))+choice(alpha))
    elif hgp_id[i]==15:
        reg_no.append('AP'+'-'+choice(alpha)+str(choice(a))+choice(alpha)+str(choice(a))+str(choice(a))+str(choice(a))+str(choice(a))+choice(alpha)+choice(alpha)+str(choice(a))+str(choice(a))+choice(alpha))
    elif hgp_id[i]==16:
        reg_no.append('TBD'+'-'+choice(alpha)+str(choice(a))+choice(alpha)+str(choice(a))+str(choice(a))+str(choice(a))+str(choice(a))+choice(alpha)+choice(alpha)+str(choice(a))+str(choice(a))+choice(alpha))
    elif hgp_id[i]==17:
        reg_no.append('BH'+'-'+choice(alpha)+str(choice(a))+choice(alpha)+str(choice(a))+str(choice(a))+str(choice(a))+str(choice(a))+choice(alpha)+choice(alpha)+str(choice(a))+str(choice(a))+choice(alpha))
    
    i+=1
len(reg_no)

In [ ]:
reg_y = random_year(1975,2020,len(h_id))
len(reg_y)

In [ ]:
a = [.3,.25,.2,.15,.1]
c = []
for i in a:
    c.append(round(len(h_id)*i))
x = 0
for i in c:
    x+=i
c

In [ ]:
ss = random_area(50,500,c[0])
s = random_area(500,1000,c[1])
m = random_area(1000,1500,c[2])
l = random_area(1500,2000,c[3])
xl = random_area(2000,2500,c[4])

In [ ]:
len(ss)+len(s)+len(m)+len(l)+len(xl)

In [ ]:
area = []
for i in ss:
    area.append(i)
for i in s:
    area.append(i)
for i in m:
    area.append(i)
for i in l:
    area.append(i)
for i in xl:
    area.append(i)
len(area)

In [ ]:
area = sample(area,len(area))
area = sample(area,len(area))
area = sample(area,len(area))

In [ ]:
home_tax = []
for i in area:
    if i <500:
        home_tax.append(choice(range(10,200)))
    elif i<1000:
        home_tax.append(choice(range(200,400)))
    elif i<1500:
        home_tax.append(choice(range(400,800)))
    elif i<2000:
        home_tax.append(choice(range(800,1000)))
    else:
        home_tax.append(choice(range(1000,1200)))
len(home_tax)

In [ ]:
water_tax = []
for i in area:
    if i <500:
        water_tax.append(choice(range(10,50)))
    elif i<1000:
        water_tax.append(choice(range(50,100)))
    elif i<1500:
        water_tax.append(choice(range(100,250)))
    elif i<2000:
        water_tax.append(choice(range(250,350)))
    else:
        water_tax.append(choice(range(350,550)))
len(water_tax)

In [ ]:
b_tax = []
for i in area:
    if i <500:
        b_tax.append(choice(range(100,250)))
    elif i<1000:
        b_tax.append(choice(range(250,500)))
    elif i<1500:
        b_tax.append(choice(range(500,750)))
    elif i<2000:
        b_tax.append(choice(range(750,1200)))
    else:
        b_tax.append(choice(range(1200,1800)))
len(b_tax)

In [ ]:
s_tax = []
for i in area:
    if i <500:
        s_tax.append(choice(range(500,1000)))
    elif i<1000:
        s_tax.append(choice(range(1000,1500)))
    elif i<1500:
        s_tax.append(choice(range(1500,2000)))
    elif i<2000:
        s_tax.append(choice(range(2000,2500)))
    else:
        s_tax.append(choice(range(2500,3000)))
len(s_tax)

In [ ]:
light_tax = []
for i in area:
    if i <500:
        light_tax.append(200)
    elif i<1000:
        light_tax.append(500)
    elif i<1500:
        light_tax.append(1500)
    elif i<2000:
        light_tax.append(2000)
    else:
        light_tax.append(2500)
len(light_tax)

In [ ]:
health_tax = []
for i in area:
    if i <500:
        health_tax.append(200)
    elif i<1000:
        health_tax.append(500)
    elif i<1500:
        health_tax.append(1500)
    elif i<2000:
        health_tax.append(2000)
    else:
        health_tax.append(2500)
len(health_tax)

In [ ]:
a = range(7)
for i in range(10):
    n_fee = []
    c = 0
    for i in area:
        if choice(a) in [0,1,3]:
            '''
            this configuration gives 40% populated'''
            c+=1
            if i <500:
                n_fee.append(10)
            elif i<1000:
                n_fee.append(30)
            elif i<1500:
                n_fee.append(70)
            elif i<2000:
                n_fee.append(110)
            else:
                n_fee.append(200)
        else:
            n_fee.append(0)
    print(c,end=' ')
len(n_fee)

In [ ]:
a = range(5)
for i in range(10):
    w_fee = []
    c = 0
    for i in area:
        if choice(a) in [0]:
            '''
            this configuration gives 20% populated'''
            c+=1
            if i <500:
                w_fee.append(20)
            elif i<1000:
                w_fee.append(60)
            elif i<1500:
                w_fee.append(100)
            elif i<2000:
                w_fee.append(170)
            else:
                w_fee.append(300)
        else:
            w_fee.append(0)
    print(c,end=' ')
len(w_fee)

In [ ]:
a = range(12)
for i in range(10):
    p_amount = []
    c = 0
    for i in area:
        if choice(a) in [0,6,5,2,4,1,3,7]:
            c+=1
            if i <500:
                p_amount.append(choice(range(200)))
            elif i<1000:
                p_amount.append(choice(range(500)))
            elif i<1500:
                p_amount.append(choice(range(1000)))
            elif i<2000:
                p_amount.append(choice(range(1600)))
            else:
                p_amount.append(choice(range(2200)))
        else:
            p_amount.append(0)
    print(c,end=' ')
len(p_amount)

In [ ]:
o_tax = []
for i in area:
    if i <500:
        o_tax.append(choice(range(50)))
    elif i<1000:
        o_tax.append(choice(range(150)))
    elif i<1500:
        o_tax.append(choice(range(300)))
    elif i<2000:
        o_tax.append(choice(range(600)))
    else:
        o_tax.append(choice(range(1200)))
len(o_tax)

# 

In [ ]:
list_birth = []

In [ ]:
b_id = []
for i in range(1,len(r_id)+1):
    b_id.append(i)
len(b_id)

In [ ]:
li = list(zip(r_id,rgp_id,rv_id,rbirth_dates,birth_place,rfirst_name,rsex,mom_id,fath_id))
li = sample(li,len(li))
li = sample(li,len(li))
li = sample(li,len(li))

In [ ]:
bres_id = []
bgp_id = []
bv_code = []
bbirth_dates = []
bbirth_place = []
bfirst_name = []
bsex  = []
bmom_id = []
bfath_id = []
for x,y,t,z,a,b,c,d,e in li:
    bres_id.append(x)
    bgp_id.append(y)
    bv_code.append(t)
    bbirth_dates.append(z)
    bbirth_place.append(a)
    bfirst_name.append(b)
    bsex.append(c)
    bmom_id.append(d)
    bfath_id.append(e)
len(bres_id)

In [ ]:
# %time
bmom_name = []
bmom_adhaar = []
bfath_name = []
bfath_adhaar = []
bmothermarriageage = []

li = list(zip(r_id,rfirst_name,radhaar))
# l2 = list((zip(bfath_id,bmom_id)))
for x in bfath_id:
    if x != 0:
        for a,b,c in li:
            if x == a:
                print(x,end=" ")
                bfath_name.append(b)
                bfath_adhaar.append(c)
                break
    else:
        bfath_name.append('')
        bfath_adhaar.append(0)
for x in bmom_id:
    if x != 0:
        for a,b,c in li:
            if x == a:
                print(x,end=" ")
                bmom_name.append(b)
                bmom_adhaar.append(c)
                bmothermarriageage.append(choice((17,18,19,20,21,22)))
                break
    else:
        bmom_name.append('')
        bmothermarriageage.append(0)
        bmom_adhaar.append(0)

In [ ]:
len(bfath_name),len(bmom_name),len(b_id),len(bfath_id),len(bmom_id)

In [ ]:
bfath_name[:10],bmom_name[:10],bfath_name[-10:],bmom_name[-10:]

In [ ]:
f_edu = [
    "Low Literacy","Low Literacy","Low Literacy","Low Literacy","Low Literacy","Low Literacy","Low Literacy","Low Literacy","Low Literacy","Low Literacy","Low Literacy","Low Literacy","Low Literacy","Low Literacy","Low Literacy","Low Literacy",
    "Under 10th","Under 10th","Under 10th","Under 10th","Under 10th",
    "10+","10+","10+","10+",
    "12+","12+","12+",
    "Bsc",
    "Higher Education",
]
m_edu = [
    "Low Literacy","Low Literacy","Low Literacy","Low Literacy","Low Literacy","Low Literacy","Low Literacy","Low Literacy","Low Literacy","Low Literacy",
    "Under 10th","Under 10th","Under 10th","Under 10th","Under 10th","Under 10th","Under 10th",
    "10+","10+","10+","10+","10+","10+","10+",
    "12+","12+","12+","12+","12+","12+","12+",
    "Bsc","Bsc","Bsc","Bsc","Bsc","Bsc","Bsc",
    "Higher Education","Higher Education","Higher Education",
]
edu = [
    "Low Literacy",
    "Under 10th",
    "10+",
    "12+",
    "Bsc",
    "Higher Education",
    '',
]

In [ ]:
bmom_edu = []
bfath_edu = []
for i in bfath_id:
    if i !=0 :
        bfath_edu.append(choice(m_edu))
    else:
        bfath_edu.append('')
for i in bmom_id:
    if i !=0 :
        bmom_edu.append(choice(f_edu))
    else:
        bmom_edu.append('')
for i in edu:
    print('mom '+i+"---"+str(bmom_edu.count(i)))
    print('father '+i+"---"+str(bfath_edu.count(i)))
len(bmom_edu),len(bfath_edu)

In [ ]:
bmom_job = []
for i in bmom_edu:
    if i == edu[0]:
        bmom_job.append('HouseWife')
    elif i == edu[1]:
        bmom_job.append('HouseWife')
    elif i == edu[2]:
        if choice(range(10)) in [1,2,3,4,5,6,7]:
            bmom_job.append('HouseWife')
        else:
            bmom_job.append('Tuitions')
    elif i == edu[3]:
        if choice(range(10)) in [1,2,3,4]:
            bmom_job.append('HouseWife')
        elif choice(range(10)) in [5,6,7]:
            bmom_job.append('Tutions')
        else:
            bmom_job.append('Junior Teacher')
    elif i == edu[4]:
        if choice(range(10)) in [1,2,3,]:
            bmom_job.append('HouseWife')
        elif choice(range(10)) in [5,6,7]:
            bmom_job.append('Tutions')
        else:
            bmom_job.append('Senior Teacher')
        
    elif i == edu[5]:
        if choice(range(10)) in [1,2]:
            bmom_job.append('HouseWife')
        elif choice(range(10)) in [3,4,5]:
            bmom_job.append('Tutions')
        elif choice(range(10)) in [6,7,8]:
            bmom_job.append('Senior Teacher')
        else:
            bmom_job.append('Company Employed')
    elif i == edu[6]:
        bmom_job.append('') 
for i in edu:
    print(i+"---"+str(bmom_edu.count(i)))
len(bmom_edu)

In [ ]:
bfath_job = []
for i in bmom_edu:
    if i == edu[0]:
        bfath_job.append('Framer')
    
    elif i == edu[1]:
        if choice(range(10)) in [1,2,3,4,5,6]:
            bfath_job.append('Farmer')
        else:
            bfath_job.append('Shop Owner')
    
    elif i == edu[2]:
        if choice(range(10)) in [1,2,3]:
            bfath_job.append('Farmer')
        else:
            bfath_job.append('Shop Owner')
    
    elif i == edu[3]:
        if choice(range(10)) in [1,2,3]:
            bfath_job.append('Farmer')
        if choice(range(10)) in [4,5,6]:
            bfath_job.append('Shop Owner')
        else:
            bfath_job.append('Small Business')
            
    elif i == edu[4]:
        if choice(range(10)) in [1,2,3]:
            bfath_job.append('Shop Owner')
        if choice(range(10)) in [1,2,3,4,5,6]:
            bfath_job.append('Small Business')
        if choice(range(10)) in [1,2,3,4,5,6]:
            bfath_job.append('Company Employed')
        
    elif i == edu[5]:
        if choice(range(10)) in [1,2,3,4,5,6]:
            bfath_job.append('Shop Owner')
        if choice(range(10)) in [1,2,3,4,5,6]:
            bfath_job.append('Small Business')
        if choice(range(10)) in [1,2,3,4,5,6]:
            bfath_job.append('Large Business')
        if choice(range(10)) in [1,2,3,4,5,6]:
            bfath_job.append('Small Comapany Owner')
    
    elif i == edu[6]:
        bfath_job.append('')  
for i in edu:
    print(i+"---"+str(bfath_edu.count(i)))
len(bfath_edu)

In [ ]:
bregi_num = []
for i in range(len(b_id)):
    bregi_num.append(str(choice(alpha))+str(choice(alpha))+str(choice(alpha))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(alpha))+str(choice(alpha)))

bRegiDate = bbirth_dates

len(bregi_num)

In [ ]:
binsert_date = []
bupdate_date = []
time = (random_time(9,18,len(b_id)))
for i in bbirth_dates:
    binsert_date.append(i+' '+time.pop())
bupdate_date = binsert_date
len(bupdate_date),len(binsert_date)

In [ ]:
bIsApprove=[]
for i in range(len(b_id)):
    c = choice(range(10))
    if c in [0]:
        bIsApprove.append(3)
    elif  c in [1,2,3]:
        bIsApprove.append(0)
    elif c in [8,9]:
        bIsApprove.append(1)
    elif c in [4,5,6,7]:
        bIsApprove.append(2)
a = [0,1,2,3]
for i in a:
    print(str(i)+'-'+str(bIsApprove.count(i)))

In [ ]:
blanks = []
for i in b_id:
    blanks.append('')
bblanks = blanks

In [ ]:
bcurraddress = bblanks
bcurrvillage = bblanks
bcurrisstayvillage = bblanks
bcurrdisrict = bblanks
bcurrstate = bblanks
bperaddress = bblanks
bperisstayvillage = bblanks
bpervillage = bblanks
bperdisrict = bblanks
bperstate = bblanks
breligion = bblanks
bchildcount = bblanks
bdeliveryplace = bblanks
bdeliverytype = bblanks
bbabyweight = bblanks
bdeliveryduration = bblanks
binformername = bblanks
binformeradd = bblanks
binformersign = bblanks
bregiorg = bblanks
bregicityvillage = bblanks
bregidistrict = bblanks
bregiremarks = bblanks
breginame = bblanks
bregicode = bblanks
bvillagename = bblanks
btalukaname = bblanks
btalukacode = bblanks
bdistrictname = bblanks
bdistrictcode = bblanks
bhandoverdate = bblanks
bhandoverby = bblanks
bhandoverbyadd = bblanks
bhandoverbysign = bblanks
bdocwrittenapp = bblanks
bdocguaranteeapp = bblanks

# 

In [ ]:
list_death = []

In [ ]:
d_id = []
for i in range(1,len(r_id)+1):
    d_id.append(i)
len(d_id)

In [ ]:
li = list(zip(r_id,rgp_id,rv_id,rbirth_dates,birth_place,rfirst_name,rsex,mom_id,fath_id,sp_id))
li = sample(li,len(li))
li = sample(li,len(li))
li = sample(li,len(li))
for i in range(len(li)-len(d_id)):
    li.pop(randrange(len(li)))
len(li)

In [ ]:
dres_id = []
dgp_id = []
dv_code = []
dbirth_dates = []
ddeath_place = []
dfirst_name = []
dsex  = []
dmom_id = []
dfath_id = []
dsp_id = []
for x,y,t,z,a,b,c,d,e,f in li:
    dres_id.append(x)
    dgp_id.append(y)
    dv_code.append(t)
    dbirth_dates.append(z)
    ddeath_place.append(a)
    dfirst_name.append(b)
    dsex.append(c)
    dmom_id.append(d)
    dfath_id.append(e)
    dsp_id.append(f)
len(dres_id)

In [ ]:
%time
dmom_name = []
dmom_adhaar = []
dfath_name = []
dfath_adhaar = []
li = list(zip(r_id,rfirst_name,radhaar))
# l2 = list((zip(bfath_id,bmom_id)))
for x in dfath_id:
    if x != 0:
        for a,b,c in li:
            if x == a:
                print(x,end=" ")
                dfath_name.append(b)
                dfath_adhaar.append(c)
                break
    else:
        dfath_name.append('')
        dfath_adhaar.append(0)
for x in dmom_id:
    if x != 0:
        for a,b,c in li:
            if x == a:
                print(x,end=" ")
                dmom_name.append(b)
                dmom_adhaar.append(c)
                break
    else:
        dmom_name.append('')
        dmom_adhaar.append(0)

In [ ]:
dsp_name = []
dsp_adhaar = []
li = list(zip(r_id,rfirst_name,radhaar))
# l2 = list((zip(bfath_id,bmom_id)))
for x in dsp_id:
    if x != 0:
        for a,b,c in li:
            if x == a:
                print(x,end=" ")
                dsp_name.append(b)
                dsp_adhaar.append(c)
                break
    else:
        dsp_name.append('')
        dsp_adhaar.append(0)

In [ ]:
x = pd.to_datetime(dbirth_dates)
death_dates = []
for i in x:
    z = random_dates(i.year,date.today().year,value_count=100)
    z = z.pop(randrange(len(z)))
    death_dates.append(z)
len(death_dates)

In [ ]:
x = pd.to_datetime(dbirth_dates).year
y = pd.to_datetime(death_dates).year
dage = []
li = list(zip(x,y))
for x,y in li:
    dage.append(y-x)
len(dage)

In [ ]:
ddeathplacedetails = []
for i in range(len(d_id)):
    ddeathplacedetails.append("Deatails Of Death")

In [ ]:
ddeathreason = []
for i in range(len(d_id)):
    ddeathreason.append("Reason of Death")

In [ ]:
alpha = []
alpha.extend(cap_alphabets)
alpha.extend(small_alphabets)
alpha = sample(alpha,len(alpha))

In [ ]:
dreginumber = []
li = list(zip(d_id,pd.to_datetime(death_dates)))
for x,y in li:
    dreginumber.append(choice(alpha)+choice(alpha)+choice(alpha)+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+'-'+str(x)+'-'+y.strftime("%m%Y"))

In [ ]:
dregidate = []
for i in pd.to_datetime(death_dates):
    try:
        i = i.replace(month = i.month+choice([1,2,3]),day = choice(range(1,28)))
        dregidate.append(i.strftime("%m/%d/%Y"))        
    except ValueError:
        try:
            i = i.replace(day = i.day+2)
            dregidate.append(i.strftime("%m/%d/%Y"))            
        except ValueError:
            dregidate.append(i.strftime("%m/%d/%Y"))


In [ ]:
dinsert_date = []
dupdate_date = []

time = random_time(9,18,len(d_id)+tcount*2)

for i in dregidate:
    dinsert_date.append(i+' '+time.pop())
dupdate_date = dinsert_date
len(dupdate_date),len(dinsert_date)

In [ ]:
dIsApprove=[]
for i in range(len(d_id)):
    c = choice(range(10))
    if c in [0]:
        dIsApprove.append(3)
    elif  c in [1,2,3]:
        dIsApprove.append(0)
    elif c in [8,9]:
        dIsApprove.append(1)
    elif c in [4,5,6,7]:
        dIsApprove.append(2)
a = [0,1,2,3]
for i in a:
    print(str(i)+'-'+str(bIsApprove.count(i)))

In [ ]:
blanks = []
for i in d_id:
    blanks.append('')
dblanks = blanks

In [ ]:
len(d_id)

In [ ]:
dIsActive = []
for i in d_id:
    if choice(range(10)) in [1,2]:
        dIsActive.append(False)
    else:
        dIsActive.append(True)

In [ ]:
dcurraddress = dblanks
dcurrvillage = dblanks
dcurrisstayvillage = dblanks
dcurrdisrict = dblanks
dcurrstate = dblanks
dperaddress = dblanks
dperisstayvillage = dblanks
dpervillage = dblanks
dperdisrict = dblanks
dperstate = dblanks
dagetype = dblanks
ddeathplacetype = dblanks
dreligion = dblanks
djobbusiness = dblanks
dmedicaltreatmenttype = dblanks
dmedicalinstitutes = dblanks
dismedicallycertified = dblanks
disdeathduringpregnancy = dblanks
dpregnancydetails = dblanks
daddictiondetails = dblanks
daddictionduration = dblanks
dinformername = dblanks
dinformeradd = dblanks
dinformersign = dblanks
dregiorg = dblanks
dregicityvillage = dblanks
dregidistrict = dblanks
dregiremarks = dblanks
dregisetrname = dblanks
dregisetrcode = dblanks
dvillagename = dblanks
dtalukaname = dblanks
dtalukacode = dblanks
ddistrictname = dblanks
ddistrictcode = dblanks
dhandoverdate = dblanks
dhandoverby = dblanks
dhandoverbyadd = dblanks
dhandoverbysign = dblanks
ddocwrittenapp = dblanks
ddocguaranteeapp = dblanks
ddocfuneralapp = dblanks

# 

In [ ]:
list_staff = []

In [ ]:
sgp_id=[]
i=[2,2,2,2,3,3,4,5,5,5]
z=0
while z<tcount:
    z+=1
    sgp_id.append(choice(i))
sgp_id.count(5),sgp_id.count(2),sgp_id.count(3),sgp_id.count(4),len(sgp_id)

In [ ]:
males = []
males_last = []
females = []
females_last = []
while len(males)<name_date_count:
    x = random_names(male,name_date_count)
    males.extend(random_first_name(x))
print('males done'+str(len(males)))

while len(males_last)<name_date_count:
    y = random_names(male,name_date_count)
    males_last.extend(random_last_name(y))
print('males_last done'+str(len(males_last)))

while len(females)<name_date_count:
    x = random_names(female,name_date_count)
    females.extend(random_first_name(x))
print('females done'+str(len(females)))

while len(females_last)<name_date_count:
    y = random_names(female,name_date_count)
    females_last.extend(random_last_name(y))
print("females_last done"+str(len(females_last)))
#------------------------------------------------------------------------------
for i in range(len(males)-tcount):
    males.pop()
print(len(males))
for i in range(len(males_last)-tcount):
    males_last.pop()
print(len(males_last))
for i in range(len(females)-tcount):
    females.pop()
print(len(females))
for i in range(len(females_last)-tcount):
    females_last.pop()
print(len(females_last))

In [ ]:
sex = []
i=0
while i<len(males):
    sex.append('Male')
    i+=1
sex1 = []
i=0
while i<len(females_last):
    sex1.append('Female')
    i+=1

In [ ]:
males = list(zip(males,males_last,sex))
females = list(zip(females,females_last,sex1))

In [ ]:
name=[]
for i in males:
    name.append(i)
for i in females:
    name.append(i)
name = sample(name,len(name))
name = sample(name,len(name))
name = sample(name,len(name))
for i in range(len(name)-tcount):
    name.pop()
    i+=1
len(name)

In [ ]:
sfirst_name=[]
slast_name=[]
ssex=[]
for x,y,z in name:
    sfirst_name.append(x)
    slast_name.append(y)
    ssex.append(z)
len(sfirst_name),len(slast_name),len(ssex)

In [ ]:
d=[]
for i in range(1,43):
    d.append(i)
i=0
x=[]
c=0
for i in range(tcount):
        c+=1
        x.append(choice(d))

In [ ]:
a=[]
for i in range(1,43):
    print(x.count(i),end='  ')

In [ ]:
designation=x
len(designation)

In [ ]:
s_id=[]
for i in range(1,tcount+1):
# for i in range(50,6050):
    s_id.append(i)
len(s_id)

In [ ]:
join_dates = random_dates(1990,2020,name_date_count+10000)
join_dates = sample(join_dates,len(join_dates))
join_dates = sample(join_dates,len(join_dates))
for i in range(len(join_dates)-tcount):
    join_dates.pop()
    i+=1
len(join_dates)

In [ ]:
end_dates,c = random_death_dates(1995,2021,value_count=name_date_count,rand=12,acceptable_list=[0,1,2,3,4,5])
for i in range(len(end_dates)-tcount):
    end_dates.pop()
    i+=1
len(end_dates),end_dates.count("")

In [ ]:
sbirth_dates = random_dates(1955,2002,name_date_count+10000)
sbirth_dates = sample(sbirth_dates,len(sbirth_dates))
sbirth_dates = sample(sbirth_dates,len(sbirth_dates))
for i in range(len(sbirth_dates)-tcount):
    sbirth_dates.pop()
    i+=1
len(sbirth_dates)

In [ ]:
sis_a=[]
for i in end_dates:
    if i == '':
        sis_a.append(True)
    else:
        sis_a.append(False)
len(sis_a)

In [ ]:
x = (0,1,2,3,4,5,6,7,8,9,)
i = 0
sadhaar = []
while i<tcount:
    sadhaar.append(str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x)))
    i+=1
len(sadhaar)

In [ ]:
a = [1,2,3,4,5,6,7,8,9,0]
i = 0
login_pass = []
while len(login_pass)<tcount:
    if choice(a)==0:
        login_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==1:
        login_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==2:
        login_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==3:
        login_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==4:
        login_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==5:
        login_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==6:
        login_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==7:
        login_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==8:
        login_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==9:
        login_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
len(login_pass)

In [ ]:
login_pass =[]
for i in range(len(sfirst_name)):
    login_pass.append(sfirst_name[i]+str(s_id[i]))
len(login_pass)

In [ ]:
slogin_pass = login_pass

# slogin_pass = []
for i in login_pass:
    slogin_pass.append(encrypt_data(i,crypto_key))
slogin_pass

In [ ]:
num = [0,1,2,3,4,5,6,7,8,9]
smobile = []
i = 0
while i<tcount:
    i+=1
    if choice(num) in [1,3,4]:
        ph = '9'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    elif choice(num) in [2,5,7]:
        ph = '7'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    elif choice(num) in [6,8,9]:
        ph = '8'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    else:
        ph = '0'+'2'+'2'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    smobile.append(int(ph))
len(smobile)

In [ ]:
n = [0,1,2,3,4,5,6,7,8,9]
semail = []
for i in rfirst_name:
    if choice(n) in [1,2,3,4,6,7,8]:
        mail = i+'@gmail.com'
    elif choice(n) in [0,9]:
        mail = i+'@yahoo.com'
    elif choice(n)==5:
        mail = i+'@outlook.com'
    else:
        mail = i+'@reddit.com'
    semail.append(mail) 
# list(zip(email,first_name))
len(semail)

In [ ]:
sadd = []
for i in s_id:
    sadd.append("Address")
len(sadd)

# 

In [ ]:
list_tax_master = []

In [ ]:
t_id = []
for i in range(99,99+len(h_id)):
    t_id.append(i)
len(t_id)

In [ ]:
gy = []
gm = []
y = [2020,2021]
m = range(1,13)
for i in range(len(h_id)):
    a = choice(y)
    s = choice(m)
    gy.append(a)
    if a == 2021 and s>5:
        gm.append(choice((1,2,3,4,5)))
    else:
        gm.append(s)

In [ ]:
fhome_tax = home_tax
flight_tax = light_tax
fhealth_tax = health_tax
fwater_tax = water_tax
fb_tax = b_tax
fs_tax = s_tax

In [ ]:
insert_date = random_dates(2019,2021,name_date_count)
time = random_time(9,18,name_date_count)
a = list(zip(insert_date,time))
tinsert_date_time = []
for i in t_id:
    for i,j in a:
        tinsert_date_time.append(i+' '+j)
        break
# for i in range(len(a)-len(h_id)):
#     tinsert_date_time.pop()
len(tinsert_date_time)

In [ ]:
total_other_tax = []
li = list(zip(flight_tax,fhealth_tax,fb_tax,fs_tax,n_fee,w_fee,o_tax))
for i,j,k,l,m,n,o in li:
    total_other_tax.append(i+j+k+l+m+n+o)
len(total_other_tax)

# 

In [ ]:
list_payment =[]

In [ ]:
p_id = []
for i in range(1,len(t_id)+1):
    p_id.append(i)
len(p_id)

In [ ]:
dbm = [4,4,1,1,1,1,2,2,2,3,3]
dbm_id = []
for i in range(len(p_id)):
    dbm_id.append(choice(dbm))
dbm_id.count(None),dbm_id.count(1),dbm_id.count(2),dbm_id.count(3),dbm_id.count(4),len(dbm_id)

In [ ]:
p_id = sample(p_id,len(p_id))

In [ ]:
a = list(zip(h_id,hgp_id,fhome_tax,fwater_tax,total_other_tax))
choice(a)
len(a)

In [ ]:
h_id1 = []
hgp_id1 = []
fhome_tax1 = []
fwater_tax1 = []
total_other_tax1 = []
a = list(zip(h_id,hgp_id,fhome_tax,fwater_tax,total_other_tax))
for x,y,z,b,c in a:
    h_id1.append(x)
    hgp_id1.append(y)
    fhome_tax1.append(z)
    fwater_tax1.append(b)
    total_other_tax1.append(c)
len(h_id1)

In [ ]:
pgp_id = []
ph_id = []
wtp_id = []
htp_id = []
otp_id = []
p_amount = []
a = list(zip(p_id,h_id1,hgp_id1,fhome_tax1,fwater_tax1,total_other_tax1,dbm_id))
for b,c,d,x,y,z,t in a:
    if t == 1:
        p_amount.append(x)
        htp_id.append(b)
        wtp_id.append(0)        
        otp_id.append(0)
    elif t == 2:
        p_amount.append(y)
        htp_id.append(0)
        wtp_id.append(b)        
        otp_id.append(0)
    elif t == 3:
        p_amount.append(z)
        htp_id.append(0)
        wtp_id.append(0)        
        otp_id.append(b)
    elif t == 4:
        p_amount.append(x+y+z)
        htp_id.append(b)
        wtp_id.append(b)        
        otp_id.append(b)
    pgp_id.append(d)
    ph_id.append(c)
a = list(zip(p_id,ph_id,pgp_id,p_amount,dbm_id))
len(a)

In [ ]:
a.sort()
p_id = []
ph_id = []
pgp_id = []
p_amount = []
dbm_id = []
for b,c,d,x,y in a:
    p_id.append(b)
    ph_id.append(c)    
    pgp_id.append(d)
    p_amount.append(x)    
    dbm_id.append(y)
t_amount = p_amount
len(pgp_id)

In [ ]:
update_time = []
d = random_dates(2019,2021,tcount)
t = random_time(9,18,tcount)
for i in range(len(pgp_id)):
    update_time.append(d[i]+' '+t[i])
update_time = sample(update_time,len(update_time))
len(update_time)#,update_time

In [ ]:
a = range(30)
pis_a = []
for i in range(len(pgp_id)):
    if choice(a) in [0]:
        pis_a.append(False)
    else:
        pis_a.append(True)
pis_a.count(True),pis_a.count(False),len(pis_a)

In [ ]:
a = [1,1,2,3,4,5,6,]
payment_mode = []
for i in range(len(pgp_id)):
    payment_mode.append(choice(a))
len(payment_mode)

In [ ]:
p_date = random_dates(2020,2021,len(pgp_id)+len(pgp_id))
for i in range(len(p_date)-len(pgp_id)):
    p_date.pop()
len(p_date)

In [ ]:
tranpay_id = p_id
tranorder_id = p_id

In [ ]:
premark = []
for i in p_id:
    premark.append('Remarks')

# 

In [ ]:
list_demand  = []

In [ ]:
deID = []
for i in range(1,len(r_id)+1):
    deID.append(i)
len(deID)

In [ ]:
deRemark = []
for i in range(len(deID)):
    deRemark.append('Remarks')

In [ ]:
deIsActive = []
deStatus = []
a = [0,1,2]
for i in range(len(deID)):
    c = choice(range(11))
    if c in [3,4,5,6,10]:
        deStatus.append(0)
        deIsActive.append(True)
    if c in [0,1,2,7]:
        deStatus.append(1)
        deIsActive.append(True)
    if c in [8,9]:
        deStatus.append(2)
        deIsActive.append(False)
for i in a:
    print(str(i)+'-'+str(deStatus.count(i)))
len(deStatus)

In [ ]:
deDemandBillMasterId = []
a = [1,2,3]
for i in range(len(deID)):
    c = choice(range(11))
    if c in [3,4,5,6,10]:
        deDemandBillMasterId.append(1)
    if c in [0,1,2,7]:
        deDemandBillMasterId.append(2)
    if c in [8,9]:
        deDemandBillMasterId.append(3)
for i in a:
    print(str(i)+'-'+str(deDemandBillMasterId.count(i)))
q = deDemandBillMasterId
len(deDemandBillMasterId)

In [ ]:
deResidentId = []
t = res_id
for i in range(len(deID)):
    deResidentId.append(choice(t))
len(deResidentId)

In [ ]:
li = list(zip(res_id,h_id,hgp_id,home_tax,water_tax,o_tax))

deAmount = []
deHouseID = []
deGramPanchayatID = []
q = list(zip(deResidentId,deDemandBillMasterId))
for i,j in q:
    for x,y,z,a,b,c in li:
         if i == x:
                if j == 1:
                    deAmount.append(a)
                    deHouseID.append(y)
                    deGramPanchayatID.append(z)
                elif j == 2:
                    deAmount.append(b)
                    deHouseID.append(y)
                    deGramPanchayatID.append(z)
                elif j == 3:
                    deAmount.append(c)
                    deHouseID.append(y)
                    deGramPanchayatID.append(z)
                break


In [ ]:
deRaiseBy = []
for i in range(len(deID)):
    deRaiseBy.append(choice(s_id))
len(deRaiseBy)

In [ ]:
deRaiseDate = random_dates(1880,date.today().year,len(deID)*2)

for i in range(len(deRaiseDate)-len(deID)):
    deRaiseDate.pop()
len(deRaiseDate)

In [ ]:
dePaidDate = []
for i,j in list(zip(deStatus,deRaiseDate)):
    if i == 1:
        try:
            paid = pd.to_datetime(j)
        except ValueError:
            deRaiseDate = [i.replace(deRaiseDate[len(dePaidDate)],'2/28/1990') for i in deRaiseDate]
            paid = pd.to_datetime('2/28/1990')
        try:
            paid = paid.replace(day = choice(range(1,28)),month = paid.month+1)
        except ValueError:
            paid = paid.replace(day = 1,month=1,year = paid.year+1)
        dePaidDate.append(paid.strftime("%m/%d/%Y"))
    else:
        dePaidDate.append('')
len(dePaidDate)

# 

In [ ]:
list_dispute = []

In [ ]:
diID = []
for i in range(1,tcount+1):
    diID.append(i)
len(diID)

In [ ]:
diStaffID = []
for i in range(len(diID)):
    diStaffID.append(choice(s_id))
len(diStaffID)

In [ ]:
diGrampanchayatID = []
for i in diStaffID:
    for x,y in list(zip(s_id,sgp_id)):
        if i == x:
            diGrampanchayatID.append(y)
            break
len(diGrampanchayatID)

In [ ]:
diSubject = []
diDetails = []
diDocuments = []
diAdminRemark = []
diResidentRemark = []
for i in range(len(diID)):
    diSubject.append('Subject')
    diDetails.append('Details')
    diDocuments.append('Documents Required')
    diAdminRemark.append('Remarks')
    diResidentRemark.append('Remarks')

In [ ]:
diIsAgainstResident = []
diResidentID = []
for i in range(len(diID)):
    c = choice(range(10))
    if c in [1,2,3,4,5,6]:
        diIsAgainstResident.append(True)
        diResidentID.append(choice(res_id))
    else:
        diIsAgainstResident.append(False)
        diResidentID.append(0)

In [ ]:
diIsActive = []
diStatus = []
diRaiseDate = []
diCloseDate = []
diUpdateTime = []
for i in range(len(diID)):
    c = choice(range(11))
    if c in [1,2]:
        diStatus.append(0)
        diIsActive.append(True)
        t = random_dates(1990,date.today().year,100).pop()
        diUpdateTime.append(t+' '+random_time(7,19,10).pop())
        diRaiseDate.append(t)
        diCloseDate.append('')
    if c in [3,4,5]:
        diStatus.append(1)
        diIsActive.append(True)
        t = random_dates(1990,date.today().year,100).pop()
        diUpdateTime.append(t+' '+random_time(7,19,10).pop())
        diRaiseDate.append(t)
        diCloseDate.append('')
    if c in [6,7,8]:
        diStatus.append(2)
        diIsActive.append(True)
        t = random_dates(1990,date.today().year,100).pop()
        diUpdateTime.append(t+' '+random_time(7,19,10).pop())
        diRaiseDate.append(t)
        diCloseDate.append('')
    if c in [9,0]:
        diStatus.append(3)
        diIsActive.append(False)
        d = random_dates(1990,date.today().year,100).pop()
        diRaiseDate.append(d)
        d1 = pd.to_datetime(d)
        try:
            d1 = d1.replace(month = d1.month+choice((1,2)),day = choice(range(1,28)))
        except ValueError:
            d1 = d1.replace(month = 1,year = d1.year+1,day = choice(range(1,28)))
        diCloseDate.append(d1.strftime('%m/%d/%Y'))
        diUpdateTime.append(d1.strftime('%m/%d/%Y')+' '+random_time(7,19,10).pop())
    if c in [10]:
        diStatus.append(4)
        diIsActive.append(False)
        d = random_dates(1990,date.today().year,100).pop()
        diRaiseDate.append(d)
        d1 = pd.to_datetime(d)
        try:
            d1 = d1.replace(month = d1.month+choice((1,2)),day = choice(range(1,28)))
        except ValueError:
            d1 = d1.replace(month = 1,year = d1.year+1,day = choice(range(1,28)))
        diCloseDate.append(d1.strftime('%m/%d/%Y'))# %H:%M:%S:%f'))
        diUpdateTime.append(d1.strftime('%m/%d/%Y')+' '+random_time(7,19,10).pop())

#

In [ ]:
list_request = []

In [ ]:
reID = []
for i in range(1,tcount+1):
    reID.append(i)
len(reID)

In [ ]:
reRemark = []
reSupportingDocument = []
for i in reID:
    reRemark.append('Remarks')
    reSupportingDocument.append('Documents')
len(reRemark),len(reSupportingDocument)

In [ ]:
reInsertDate = []
while len(reInsertDate)<len(reID):
    x = random_dates(1990,date.today().year,len(reID))
    reInsertDate.extend(x)
for i in range(len(reInsertDate)-len(reID)):
    reInsertDate.pop(randrange(len(reInsertDate)))
len(reInsertDate)

In [ ]:
reHouseID = []
reResidentID = []
reGrampanchayatID = []

li = list(zip(h_id,res_id,rgp_id))
li = sample(li,len(li))
li = sample(li,len(li))
li = sample(li,len(li))
for i in reID:
    x,y,z = choice(li)
    reHouseID.append(x)
    reResidentID.append(y)
    reGrampanchayatID.append(z)
len(reHouseID)

In [ ]:
reRequestFor = []
a = [1,2,3,4,5,6]
for i in reID:
    reRequestFor.append(choice(a))
len(reRequestFor)

In [ ]:
reRequestID = []
for x,y,z in list(zip(reRequestFor,reResidentID,reID)):
    if x in [1,2,3]:
        reRequestID.append(z)
    else:
        reRequestID.append(y)
len(reRequestID)

In [ ]:
reIsApprove = []
a = [0,0,1,1,1,2,2,2,2,3]
for i in reID:
    reIsApprove.append(choice(a))
len(reIsApprove)

In [ ]:
reStaffApproveID = []
reStaffApproveDate = []
reAdminApproveDate = []
reAdminApproveID = []
reIsActive = []
reUpdateDateTime = []
li = list(zip(s_id,sgp_id))
for i,j in list(zip(reIsApprove,reInsertDate)):
    if i == 0:
        reStaffApproveID.append(0)
        reStaffApproveDate.append('')
        reAdminApproveID.append(0)
        reAdminApproveDate.append('')
        reIsActive.append(True)
        reUpdateDateTime.append(j+' '+random_time(7,19,10).pop())
    if i == 1:
        j = pd.to_datetime(j)
        try:
            j = j.replace(month = j.month+choice([0,1]),day = choice(range(1,28)))
        except ValueError:
            j = j.replace(month = 1,day = 1,year = j.year+1)
        j = j.strftime('%m/%d/%Y')
        reStaffApproveID.append(choice(s_id))
        reStaffApproveDate.append(j)
        reAdminApproveID.append(0)
        reAdminApproveDate.append('')
        reIsActive.append(True)
        reUpdateDateTime.append(j+' '+random_time(7,19,10).pop())
    if i == 2:
        j = pd.to_datetime(j)
        try:
            j = j.replace(month = j.month+choice([0,1]),day = choice(range(1,28)))
        except ValueError:
            j = j.replace(month = 1,day = 1,year = j.year+1)
        j = j.strftime('%m/%d/%Y')
        reStaffApproveID.append(choice(s_id))
        reStaffApproveDate.append(j)
        j = pd.to_datetime(j)
        try:
            j = j.replace(month = j.month+choice([0,1]),day = choice(range(1,28)))
        except ValueError:
            j = j.replace(month = 1,day = 1,year = j.year+1)
        j = j.strftime('%m/%d/%Y')
        reAdminApproveID.append(choice(s_id))
        reAdminApproveDate.append(j)
        reIsActive.append(True)
        reUpdateDateTime.append(j+' '+random_time(7,19,10).pop())
    if i == 3:
        if choice((0,1))==0:
            j = pd.to_datetime(j)
            try:
                j = j.replace(month = j.month+choice([0,1]),day = choice(range(1,28)))
            except ValueError:
                j = j.replace(month = 1,day = 1,year = j.year+1)
            j = j.strftime('%m/%d/%Y')
            reStaffApproveID.append(choice(s_id))
            reStaffApproveDate.append('')
            reAdminApproveID.append(0)
            reAdminApproveDate.append('')
            reIsActive.append(False)
            reUpdateDateTime.append(j+' '+random_time(7,19,10).pop())
        if choice((0,1))==1:
            j = pd.to_datetime(j)
            try:
                j = j.replace(month = j.month+choice([0,1]),day = choice(range(1,28)))
            except ValueError:
                j = j.replace(month = 1,day = 1,year = j.year+1)
            j = j.strftime('%m/%d/%Y')
            reStaffApproveID.append(choice(s_id))
            reStaffApproveDate.append(j)
            j = pd.to_datetime(j)
            try:
                j = j.replace(month = j.month+choice([0,1]),day = choice(range(1,28)))
            except ValueError:
                j = j.replace(month = 1,day = 1,year = j.year+1)
            j = j.strftime('%m/%d/%Y')
            reAdminApproveID.append(choice(s_id))
            reAdminApproveDate.append(j)
            reIsActive.append(False)
            reUpdateDateTime.append(j+' '+random_time(7,19,10).pop())
len(reUpdateDateTime),len(reIsActive),len(reAdminApproveDate),len(reAdminApproveID),len(reUpdateDateTime),

# 

In [ ]:
a = range(10)
is_a = []
for i in range(tcount):
    if choice(a) in [0]:
        is_a.append(False)
    else:
        is_a.append(True)
is_a.count(True),is_a.count(False),len(is_a)

ris_a = sample(is_a,len(is_a))
list_resident = list(zip(r_id,rgp_id,title,rfirst_name,rlast_name,fath_id,
                         mom_id,rsex,rbirth_dates,birth_place,rh_id,radhaar,
                         radd,rlogin_pass,ris_a,marital,disability,sp_id,
                         sp_name,father,mother,mid_name,rmobile,remail,
                         rv_id))

his_a = sample(is_a,len(is_a))
a = add
for i in range(len(add)-len(h_id)):
    is_a.pop()
    a.pop()
list_house = list(zip(h_id,hgp_id,res_id,pr_no,reg_no,reg_y,area,
                      home_tax,light_tax,health_tax,water_tax,b_tax,
                      s_tax,n_fee,w_fee,o_tax,a,his_a,hv_id))

tis_a = sample(is_a,len(is_a))
list_tax_master = list(zip(t_id,hgp_id,res_id,h_id,gy,gm,fhome_tax,
                          flight_tax,fhealth_tax,fwater_tax,fb_tax,fs_tax,
                          n_fee,w_fee,o_tax,add,tis_a,tinsert_date_time,#t_amount,
                           total_other_tax,htp_id,wtp_id,otp_id))

list_staff = list(zip(s_id,designation,join_dates,end_dates,sfirst_name,
                      slast_name,ssex,sbirth_dates,sadhaar,sadd,sis_a,
                      slogin_pass,semail,smobile,sgp_id))

list_payment = list(zip(p_id,p_date,ph_id,p_amount,payment_mode,pis_a,update_time,tranpay_id,tranorder_id,premark,pgp_id,dbm_id))


list_birth =list(zip(
    b_id,bres_id,bgp_id,bbirth_dates,bbirth_place,bfirst_name,bsex,bmom_name,bmom_adhaar,bmom_edu,bmom_job,bmothermarriageage,bfath_name,bfath_adhaar,
    bfath_edu,bfath_job,bcurraddress,bcurrvillage,bcurrisstayvillage,bpervillage,bperdisrict,bperstate,breligion,bchildcount,bbirth_place,
    bdeliverytype,bbabyweight,bdeliveryduration,binformername,binformeradd,binformersign,bregi_num,binsert_date,bregiorg,bregicityvillage,bregidistrict,
    bregiremarks,breginame,bregicode,bvillagename,bv_code,btalukaname,btalukacode,bdistrictname,bdistrictcode,bhandoverdate,bhandoverby,bhandoverbyadd,
    bhandoverbysign,bdocwrittenapp,bdocguaranteeapp,bIsApprove,bmom_id,bfath_id,binsert_date,bupdate_date
))

list_death = list(zip(d_id,dres_id,dgp_id,death_dates,dfirst_name,dmom_name,dmom_adhaar,dfath_name,dfath_id,dsp_name,dsp_adhaar,dcurraddress,dcurrvillage,
                     dcurrisstayvillage,dcurrdisrict,dcurrstate,dperaddress,dperisstayvillage,dpervillage,dperdisrict,dperstate,dage,dagetype,ddeathplacetype,
                     ddeathplacedetails,dreligion,djobbusiness,dmedicaltreatmenttype,dmedicalinstitutes,dismedicallycertified,ddeathreason,disdeathduringpregnancy,
                      dpregnancydetails,daddictiondetails,daddictionduration,dinformername,dinformeradd,dinformersign,dreginumber,dregidate,dregiorg,dregicityvillage,
                      dregidistrict,dregiremarks,dregisetrname,dregisetrcode,dvillagename,dv_code,dtalukaname,dtalukacode,ddistrictname,ddistrictcode,dhandoverdate,
                      dhandoverby,dhandoverbyadd,dhandoverbysign,ddocwrittenapp,ddocguaranteeapp,ddocfuneralapp,dIsApprove,dIsActive,dinsert_date,dupdate_date                      
                     ))

list_demand = list(zip(deID,deDemandBillMasterId,deResidentId,deRaiseBy,deStatus,deAmount,deRaiseDate,dePaidDate,deRemark,deGramPanchayatID,deIsActive,
                      deHouseID))

list_request = list(zip(reID,reResidentID,reGrampanchayatID,reRequestFor,reInsertDate,reIsApprove,reIsActive,reRemark,reHouseID,reSupportingDocument,
                        reRequestID,reStaffApproveID,reStaffApproveDate,reAdminApproveID,reAdminApproveDate,reUpdateDateTime))

list_dispute = list(zip(diID,diGrampanchayatID,diIsAgainstResident,diSubject,diDetails,diResidentID,diStaffID,diDocuments,diIsActive,diStatus,
                        diRaiseDate,diCloseDate,diAdminRemark,diResidentRemark,diUpdateTime))

In [ ]:
c = 1
print('resident')
for i in (r_id,rgp_id,title,rfirst_name,rlast_name,fath_id,
                         mom_id,rsex,rbirth_dates,birth_place,rh_id,radhaar,
                         radd,rlogin_pass,ris_a,marital,disability,sp_id,
                         sp_name,father,mother,mid_name,rmobile,remail,
                         rv_id):
    print(str(c) +'-'+str(len(i)),end=' ')
    c+=1

In [ ]:
print('\nhouse')
c = 1
for i in (h_id,hgp_id,res_id,pr_no,reg_no,reg_y,area,
                      home_tax,light_tax,health_tax,water_tax,b_tax,
                      s_tax,n_fee,w_fee,o_tax,a,his_a,hv_id):
    print(str(c) +'-'+str(len(i)),end=' ')
    c+=1

In [ ]:
print('\ntax master')
c = 1
for i in (t_id,hgp_id,res_id,h_id,gy,gm,fhome_tax,
                          flight_tax,fhealth_tax,fwater_tax,fb_tax,fs_tax,
                          n_fee,w_fee,o_tax,add,tis_a,tinsert_date_time,a,
                           total_other_tax,htp_id,wtp_id,otp_id):
    print(str(c) +'-'+str(len(i)),end=' ')
    c+=1

In [ ]:
print('\nstaff')
c = 1
for i in (s_id,designation,join_dates,end_dates,sfirst_name,
                      slast_name,ssex,sbirth_dates,sadhaar,sadd,sis_a,
                      slogin_pass,semail,smobile,sgp_id):
    print(str(c) +'-'+str(len(i)),end=' ')
    c+=1

In [ ]:
print('\npayment')
c = 1
for i in (p_id,p_date,ph_id,p_amount,payment_mode,pis_a,update_time,tranpay_id,tranorder_id,premark,pgp_id,dbm_id):
    print(str(c) +'-'+str(len(i)),end=' ')
    c+=1

In [ ]:
print('\nBirth Master')
c = 1
for i in (b_id,bres_id,bgp_id,bbirth_dates,bbirth_place,bfirst_name,bsex,bmom_name,bmom_adhaar,bmom_edu,bmom_job,bmothermarriageage,bfath_name,bfath_adhaar,
    bfath_edu,bfath_job,bcurraddress,bcurrvillage,bcurrisstayvillage,bpervillage,bperdisrict,bperstate,breligion,bchildcount,bbirth_place,
    bdeliverytype,bbabyweight,bdeliveryduration,binformername,binformeradd,binformersign,bregi_num,binsert_date,bregiorg,bregicityvillage,bregidistrict,
    bregiremarks,breginame,bregicode,bvillagename,bv_code,btalukaname,btalukacode,bdistrictname,bdistrictcode,bhandoverdate,bhandoverby,bhandoverbyadd,
    bhandoverbysign,bdocwrittenapp,bdocguaranteeapp,bIsApprove,bmom_id,bfath_id,binsert_date,bupdate_date):
    print(str(c) +'-'+str(len(i)),end=' ')
    c+=1

In [ ]:
print('\nDeath Master')
c = 1
for i in (d_id,dres_id,dgp_id,death_dates,dfirst_name,dmom_name,dmom_adhaar,dfath_name,dfath_id,dsp_name,dsp_adhaar,dcurraddress,dcurrvillage,
                     dcurrisstayvillage,dcurrdisrict,dcurrstate,dperaddress,dperisstayvillage,dpervillage,dperdisrict,dperstate,dage,dagetype,ddeathplacetype,
                     ddeathplacedetails,dreligion,djobbusiness,dmedicaltreatmenttype,dmedicalinstitutes,dismedicallycertified,ddeathreason,disdeathduringpregnancy,
                      dpregnancydetails,daddictiondetails,daddictionduration,dinformername,dinformeradd,dinformersign,dreginumber,dregidate,dregiorg,dregicityvillage,
                      dregidistrict,dregiremarks,dregisetrname,dregisetrcode,dvillagename,dv_code,dtalukaname,dtalukacode,ddistrictname,ddistrictcode,dhandoverdate,
                      dhandoverby,dhandoverbyadd,dhandoverbysign,ddocwrittenapp,ddocguaranteeapp,ddocfuneralapp,dIsApprove,dIsActive,dinsert_date,dupdate_date                      
         ):
    print(str(c) +'-'+str(len(i)),end=' ')
    c+=1

In [ ]:
print('\nDemand Master')
c = 1
for i in (deID,deDemandBillMasterId,deResidentId,deRaiseBy,deStatus,deAmount,deRaiseDate,dePaidDate,deRemark,deGramPanchayatID,deIsActive,
                      deHouseID
         ):
    print(str(c) +'-'+str(len(i)),end=' ')
    c+=1

In [ ]:
print('\nRequest Master')
c = 1
for i in (reID,reResidentID,reGrampanchayatID,reRequestFor,reInsertDate,reIsApprove,reIsActive,reRemark,reHouseID,reSupportingDocument,
                        reRequestID,reStaffApproveID,reStaffApproveDate,reAdminApproveID,reAdminApproveDate,reUpdateDateTime
         ):
    print(str(c) +'-'+str(len(i)),end=' ')
    c+=1

In [ ]:
print('\nDispute Master')
c = 1
for i in (diID,diGrampanchayatID,diIsAgainstResident,diSubject,diDetails,diResidentID,diStaffID,diDocuments,diIsActive,diStatus,
                        diRaiseDate,diCloseDate,diAdminRemark,diResidentRemark,diUpdateTime
         ):
    print(str(c) +'-'+str(len(i)),end=' ')

    c+=1

In [ ]:
lists = [
    list_staff,
    list_resident,
    list_house,
    list_birth,
    list_death,
    list_tax_master,
    list_payment,
    list_request,
    list_demand,
    list_dispute,
]
tables = [
    'Staff',
    'Resident',
    'House',
    'BirthMaster',
    'DeathMaster',
    'TaxMaster',
    'Payment',
    'RequestCertificate',
    'DemandBillDetails',
    'DisputeMaster',
]
li = list(zip(lists,tables))
for x,y in li:
    print(y+" "+ str(len(x)))

# 

In [ ]:
database_table_list(db)

# 

# Export data to the database

In [ ]:
a = range(10)
is_a = []
for i in range(tcount):
    if choice(a) in [0]:
        is_a.append(False)
    else:
        is_a.append(True)
is_a.count(True),is_a.count(False),len(is_a)

ris_a = sample(is_a,len(is_a))
list_resident = list(zip(#r_id,
    rgp_id,title,rfirst_name,rlast_name,fath_id,
                         mom_id,rsex,rbirth_dates,birth_place,rh_id,radhaar,
                         radd,rlogin_pass,ris_a,marital,disability,sp_id,
                         sp_name,father,mother,mid_name,rmobile,remail,
                         rv_id))

his_a = sample(is_a,len(is_a))
a = add
for i in range(len(add)-len(h_id)):
    is_a.pop()
    a.pop()
list_house = list(zip(#h_id,
    hgp_id,res_id,pr_no,reg_no,reg_y,area,
                      home_tax,light_tax,health_tax,water_tax,b_tax,
                      s_tax,n_fee,w_fee,o_tax,a,his_a,hv_id))

tis_a = sample(is_a,len(is_a))
list_tax_master = list(zip(#t_id,
    hgp_id,res_id,h_id,gy,gm,fhome_tax,
                          flight_tax,fhealth_tax,fwater_tax,fb_tax,fs_tax,
                          n_fee,w_fee,o_tax,add,tis_a,tinsert_date_time,t_amount,
                           total_other_tax,htp_id,wtp_id,otp_id))

list_staff = list(zip(#s_id,
    designation,join_dates,end_dates,sfirst_name,
                      slast_name,ssex,sbirth_dates,sadhaar,sadd,sis_a,
                      slogin_pass,semail,smobile,sgp_id))

list_payment = list(zip(#p_id,
    p_date,ph_id,p_amount,payment_mode,pis_a,update_time,tranpay_id,tranorder_id,premark,pgp_id,dbm_id))


list_birth =list(zip(#b_id,
    bres_id,bgp_id,bbirth_dates,bbirth_place,bfirst_name,bsex,bmom_name,bmom_adhaar,bmom_edu,bmom_job,bmothermarriageage,bfath_name,bfath_adhaar,
    bfath_edu,bfath_job,bcurraddress,bcurrvillage,bcurrisstayvillage,bcurrdisrict,bcurrstate,bperaddress,bperisstayvillage,bpervillage,bperdisrict,bperstate,breligion,bchildcount,bbirth_place,
    bdeliverytype,bbabyweight,bdeliveryduration,binformername,binformeradd,binformersign,bregi_num,binsert_date,bregiorg,bregicityvillage,bregidistrict,
    bregiremarks,breginame,bregicode,bvillagename,bv_code,btalukaname,btalukacode,bdistrictname,bdistrictcode,bhandoverdate,bhandoverby,bhandoverbyadd,
    bhandoverbysign,bdocwrittenapp,bdocguaranteeapp,bIsApprove,bmom_id,bfath_id,binsert_date,bupdate_date
))

list_death = list(zip(#d_id,
    dres_id,dgp_id,death_dates,dfirst_name,dmom_name,dmom_adhaar,dfath_name,dfath_id,dsp_name,dsp_adhaar,dcurraddress,dcurrvillage,
                     dcurrisstayvillage,dcurrdisrict,dcurrstate,dperaddress,dperisstayvillage,dpervillage,dperdisrict,dperstate,dage,dagetype,ddeathplacetype,
                     ddeathplacedetails,dreligion,djobbusiness,dmedicaltreatmenttype,dmedicalinstitutes,dismedicallycertified,ddeathreason,disdeathduringpregnancy,
                      dpregnancydetails,daddictiondetails,daddictionduration,dinformername,dinformeradd,dinformersign,dreginumber,dregidate,dregiorg,dregicityvillage,
                      dregidistrict,dregiremarks,dregisetrname,dregisetrcode,dvillagename,dv_code,dtalukaname,dtalukacode,ddistrictname,ddistrictcode,dhandoverdate,
                      dhandoverby,dhandoverbyadd,dhandoverbysign,ddocwrittenapp,ddocguaranteeapp,ddocfuneralapp,dIsApprove,dIsActive,dinsert_date,dupdate_date                      
                     ))

list_demand = list(zip(#deID,
    deDemandBillMasterId,deResidentId,deRaiseBy,deStatus,deAmount,deRaiseDate,dePaidDate,deGramPanchayatID,deRemark,deIsActive,
                      deHouseID))

list_request = list(zip(#reID,
    reResidentID,reGrampanchayatID,reRequestFor,reInsertDate,reIsApprove,reIsActive,reRemark,reHouseID,reSupportingDocument,
                        reRequestID,reStaffApproveID,reStaffApproveDate,reAdminApproveID,reAdminApproveDate,reUpdateDateTime))

list_dispute = list(zip(#diID,
    diGrampanchayatID,diIsAgainstResident,diSubject,diDetails,diResidentID,diStaffID,diDocuments,diIsActive,diStatus,
                        diRaiseDate,diCloseDate,diAdminRemark,diResidentRemark,diUpdateTime))


In [ ]:
tables = [
    'Staff',
    'Resident',
    'House',
    'BirthMaster',
    'DeathMaster',
    'TaxMaster',
    'Payment',
    'RequestCertificate',
    'DemandBillDetails',
    'DisputeMaster',
]
len(tables)

In [ ]:
lists = [
    list_staff,
    list_resident,
    list_house,
    list_birth,
    list_death,
    list_tax_master,
    list_payment,
    list_request,
    list_demand,
    list_dispute,
]
len(lists)

In [ ]:
masters = [
    'GramPanchayat',
    'Village',
    'PhysDisableStatus',
    'MaritalStatus',
    'PaymentMethod',
    'Designation',
    'DemandBillMaster',
    'RequestMaster',   
]

In [ ]:
con = connect(db)
cursor = con.cursor()
con.autocommit = False

In [ ]:
for a in masters:

    df = pd.read_csv('./{}.csv'.format(a),index_col=0)
    try:
        df = df.drop(index='ID',axis=1)
    except KeyError:
        pass
    df = df.values.tolist()

    li = df

    client = get_client(key)
    sheet = client.open(sheet_name)

    wb = sheet.worksheet(a)

    df = wb.get_all_values()

    b = df.pop(0)

    b.pop(0)

    c=''
    v=''
    x = 1
    for i in b:
        if x == len(b):
            c = c+i
            v = v+'?'
        else:
            c = c+i+','
            v = v+'?,'
        x+=1

    sql = "insert into {} ({}) values({})"

    print(sql.format(a,c,v))

    cursor.executemany(sql.format(a,c,v),li)

    con.commit()

In [ ]:
q = 0
for a in tables:
    client = get_client(key)
    sheet = client.open(sheet_name)

    wb = sheet.worksheet(a)

    df = wb.get_all_values()

    b = df.pop(0)

    b.pop(0)

    c=''
    v=''
    x = 1
    for i in b:
        if x == len(b):
            c = c+i
            v = v+'?'
        else:
            c = c+i+','
            v = v+'?,'
        x+=1

    sql = "insert into {} ({}) values({})"

    print(sql.format(a,c,v))

    cursor.executemany(sql.format(a,c,v),lists[q])

    con.commit()
    print(a+' Done')
    q+=1

In [ ]:
con.close()

# 

# Rough

In [ ]:
    names = ['TaxMaster','Payment','Staff']
    name = tables[0]
# for name in names:
    print(name)
    wb = sheet.worksheet(name)
    df = wb.get_all_values()
    print('Fetched Sheets DataFrame')
    cols = []
    c = 0
    b=[]
#     for i in df.columns:
#         cols.append(i)
    cols = df.pop(0)
    cols.pop(0)
    cols.pop(-4)
#     df = df[cols]
    print('Column list ready')
#     a = sheets_to_datalist(sheet_name,name)
    a = list_tax_master
    print('Fetched Datalist')
    for i in range(len(a)):
        a[i]= list(a[i])
        a[i].pop(0)
#         a[i].pop(0)
#         a[i].pop(-4)
    c = ""
    v = ""
    x = 1
    for i in cols:
        if x == len(cols):
            c = c+i
            v = v+'?'
        else:
            c = c+i+','
            v = v+'?,'
        x+=1
    sql = "insert into {} ({}) values({})"
    print(sql.format(name,c,v),'\nExecuting statement')
    cursor.executemany(sql.format(name,c,v),a)
    cursor.commit()
    print(name,' : Done\n')

In [ ]:
con = connect(db)
cursor = con.cursor()
con.autocommit = False

In [ ]:
import datetime
datetime.datetime.strptime(a[0][0], '%m/%d/%Y').isoformat()

In [ ]:
con.close()

In [ ]:
cursor.commit()

# 

# Run Before the whole code
## Run the encryption cell in the beginning After this
### Then run the whole code

In [ ]:
tables = [
    'GramPanchayat',
    'Village',
    'Resident',
    'BirthMaster',
    'DeathMaster',
    'PhysDisableStatus',
    'MaritalStatus',
#     'marriage',
    'House',
    'TaxMaster',
    'PaymentMethod',
    'Payment',
    'Designation',
    'Staff',
    'DemandBillMaster',
#     'OccasionMaster',
    'RequestMaster',
    'RequestCertificate',
#     'OccasionDetails',
    'DemandBillDetails',
    'DisputeMaster',
    'RequestCertificate',
#     'NoticeBoard',
#     'EventManaging',
         ]

In [ ]:
sheet_empty(sheet_name)
sheets_import_colums(tables,db,sheet_name)

# li = [
    'GramPanchayat',
    'Staff',
    'DeathMaster',
    'Resident',
    'House',
]

# for i in li:
    wb = sheet.worksheet(i)
    df = wb.get_all_values()
    wb.delete_rows(2,len(df))

# wb = sheet.worksheet('Designation')

wb.delete_rows(5,8)

cell_list = wb.range('a2:a43')

cell_values = range(1,43)

for i,val in enumerate(cell_values):
    cell_list[i].value = val

wb.update_cells(cell_list)

# wb = sheet.worksheet('DemandBillMaster')
cell_list = wb.range('a2:a4')
cell_values = [1,2,3]
for i, val in enumerate(cell_values):
    cell_list[i].value = val
wb.update_cells(cell_list)

# li = [
    'GramPanchayat',
    'Village',
]

# for i in li:
    client = get_client(key)
    sheet = client.open(sheet_name)
    try:
        wb = sheet.worksheet(i)
        df = wb.get_all_values()
        df = df[2]
    except:
        df = pd.read_csv('./{}.csv'.format(i))

        df = df.values

        df = df.tolist()

        wb = sheet.worksheet(i)

        wb.append_rows(df)

# 

# Dont run if database is empty

This is to create csv files from the data imported in the sheet from the database

So the database needs to have some values for these tables else it will create or replace the csv with a blank csv file

In [ ]:
masters = [
    'PhysDisableStatus',
    'MaritalStatus',
    'PaymentMethod',
    'Designation',
    'DemandBillMaster',
    'RequestMaster',   
]

In [ ]:
client = get_client(key)

sheet = client.open(sheet_name)

for i in masters:
    wb = sheet.worksheet(i)
    df = wb.get_all_values()
    df = pd.DataFrame(df)
    df.to_csv('./{}.csv'.format(i),index=False)
    print(i+' CSV Done')

# 